Here are some more side tests to clarify / justify details, that would clutter the main test notebook.

In [ ]:
from importlib import reload
import helper as hlp

import numpy as np
import numpy.lib.recfunctions

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm, Normalize
%matplotlib inline

import scipy.interpolate as sci
import scipy.optimize as sco
import scipy.integrate as scint
import scipy.stats as scs
import scipy.signal as scsignal

from tqdm import tqdm_notebook as tqdm
import json
import datetime
import pickle
from astropy.time import Time as astrotime
from corner import corner

import sklearn.neighbors as skn
import sklearn.model_selection as skms  # Newer version of grid_search

from corner_hist import corner_hist
from anapymods3.plots.general import (split_axis, get_binmids,
                                      hist_marginalize, dg)
from anapymods3.stats.sampling import rejection_sampling
from anapymods3.general.misc import (fill_dict_defaults,
                                     flatten_list_of_1darrays)

import tdepps.bg_injector as BGInj
import tdepps.bg_rate_injector as BGRateInj
import tdepps.rate_function as RateFunc
import tdepps.llh as LLH
import tdepps.analysis as Analysis

secinday = 24. * 60. * 60.

# Load data

Load IC86 data from epinat, which should be the usual IC86-I (2011) PS sample, but pull corrected and OneWeights corrected by number of events generated.

In [ ]:
exp = np.load("data/IC86_I_data.npy")
mc = np.load("data/IC86_I_mc.npy")
# Use the officially stated livetime, not the ones from below
livetime = 332.61

# Data livetime comparison to v1.4

Let's compare to the v1.4 list, as used by jfeintzig.
Oddly we have 0.2 days less livetime as he had.
The number of runs is correct though

In [ ]:
# New livetime from iclive
run_list = hlp.get_run_list()
run_dict = hlp.get_run_dict(run_list)
inc_run_arr, ic_livetime = hlp.get_good_runs(run_dict)

print("Total runs from iclive     : ", len(inc_run_arr))
print("IC86-I livetime from iclive: ", ic_livetime)

In [ ]:
# For comparison, also parse the v1.4 list
# Should be: 1081 runs, with a total livetime of 332.61 days.
with open("data/Prelim_IC86-I_v1.4a.txt",'r') as f:
    data = []
    for line in f.readlines():
        data.append(line.replace('\n',''))
        
# Skip to beginning of run info
data = data[73:]

# Split at white space
data = [d.split() for d in data]

dtype = [("runID", np.int), ("duration", np.float), ("IT", "|S2"),
         ("CONF", "|S7"), ("FLAG", "|S6")]
runlist = np.empty((len(data),), dtype=dtype)

runlist["runID"] = np.array([int(d[0]) for d in data])
runlist["duration"] = np.array([float(d[3]) for d in data])
runlist["IT"] = np.array([d[5] for d in data])
runlist["CONF"] = np.array([d[6] for d in data])
runlist["FLAG"] = np.array([d[7] for d in data])

# Now filter: Include IT=it, CONF=full, FLAG=GOOD, exclude strange rate runs
exclude_rate = [120028, 120029, 120030, 120087, 120156, 120157]
itgood = runlist["IT"] == b"IT"  # Somehow only bitwise comparison is non-empty
confgood = runlist["CONF"] == b"full"
flaggood = runlist["FLAG"] == b"GOOD"
ratebad = np.in1d(runlist["runID"], exclude_rate)

include = itgood & confgood & flaggood & ~ratebad
runlist_inc = runlist[include]

# Get the livetime of the sample in days
hoursindays = 24.
secinday = hoursindays * 60. * 60.
old_livetime = np.sum(runlist_inc["duration"]) / hoursindays

print("Total runs from v1.4     : ", len(runlist_inc))
print("Total livetime from v1.4 : ", old_livetime)

Let's see, if the 120 extra runs in the new runlist make up for the difference of about 10 days in livetime.

In [ ]:
iclive_in_old = np.in1d(inc_run_arr["runID"], runlist_inc["runID"])
not_in_old = inc_run_arr[~iclive_in_old]

start = not_in_old["start_mjd"]
stop = not_in_old["stop_mjd"]
missing_livetime = np.sum(stop - start)

print("\nOfficial IC86-I PS livetime: ", livetime)
print("Total livetime from v1.4   : ", old_livetime)
print("IC86-I livetime from iclive: ", ic_livetime)

print("\nMissing runs in old: ", len(not_in_old))
print("Livetime icliv - old :", ic_livetime - old_livetime)

print("\nDiff from summing missing runs           : ", missing_livetime)
print("New iclive livetime with same runs as old: ",
      ic_livetime - missing_livetime)

print("\nTotal rate [Hz] over total livetime: ",
      len(exp) / (livetime * secinday))

All runs from the new run list that zero events, make up for the missing runs in the old runlist, so this is consisting.

Dont't know though, where the missing 0,2 days come from. Probably some runtimes have shifted a little making some extra livetime in the new list.

In [ ]:
# Store events in bins with run borders
exp_times = exp["timeMJD"]
start_mjd = inc_run_arr["start_mjd"]
stop_mjd = inc_run_arr["stop_mjd"]

tot = 0
evts_in_run = {}
for start, stop , runid in zip(start_mjd, stop_mjd, inc_run_arr["runID"]):
    mask = (exp_times >= start) & (exp_times < stop)
    evts_in_run[runid] = exp[mask]
    tot += np.sum(mask)
    
# Crosscheck, if we got all events and counted nothing double
print("Do we have all events? ", tot == len(exp))
print("  Events selected : ", tot)
print("  Events in exp   : ", len(exp))

# Create binmids and histogram values in each bin
binmids = 0.5 * (start_mjd + stop_mjd)
h = np.zeros(len(binmids), dtype=np.float)

for i, evts in enumerate(evts_in_run.values()):
    h[i] = len(evts)
    
m = (h > 0)
print("Runs with 0 events :", np.sum(~m))
print("Runtime in those runs: ", np.sum(inc_run_arr["stop_mjd"][~m] -
                                        inc_run_arr["start_mjd"][~m]))

# Remove all zero event runs (artifacts from new run list) and calc the rate
stop_mjd, start_mjd = stop_mjd[m], start_mjd[m]
h = h[m] / ((stop_mjd - start_mjd) * secinday)
binmids = binmids[m]

# Time dependent rate function

**Note: I think it is unnecessary to use a time and declination dependent rate. The spatial part is injected from the data BG from KDE anyways. So we just need to have the rate to determine how much events we inject allsky.**

Rate ist time dependent because of seasonal variation.
We take this varariation into account by fitting a priodic function to the time resolved rate.

The data is built by calculating the rate in each run as seen before.
This rate is correctly normalized and smoothes local fluctuations.

### Peridoc function with a weighted least squares fit

See side_test for comparison to spline fits.
The function is a simple sinus scalable by 4 parameters to fit the shape of the rates:

$$
    f(x) = a\cdot \sin(b\cdot(x - c)) + d
$$

The least squares loss function is

$$
    R = \sum_i (w_i(y_i - f(x_i)))^2
$$

Weights are standard deviations from poisson histogram error.

$$
    w_i = \frac{1}{\sigma_i}
$$

Seed values are estimated from plot rate vs time.

- Period should be 365 days (MJD) because we have one year of data so we choose $b0 = 2\pi/365$.
- Amplitude is about $a_0=-0.0005$, because sinus seems to start with negative values.
- The x-offset is choose as the first start date, to get the right order of magnitude.
- The y-axis intersection $d$ schould be close to the weighted average, so we take this as a seed.

The bounds are motivated as follows (and if we don't hit them, it's OK to use them).

- Amplitude $a$ should be positive, this also resolves a degenracy between a-axis offset.
- The period $b$ should scatter around one year, a period larger than +-1 half a year is unphysical.
- The x-offset $c$ cannot be greater than the initial +- the period because we have a periodic function.
- The y-axis offset $d$ is arbitrarily constrained, but as seen from the plot it should not exceed 0.1. 

## Proposed was something like this

Rate ist time dependent because of seasonal variation and delination dependent because the detector acceptance is declination dependent.
A correletation should not exist or be very small.

So we express the rate in depence of time and decliantion as

$$
    R(t,\delta) = R_T(t)\cdot R_\delta(t)
$$

with independent parts in declination and time each.

The time parts is constructed by fitting a periodic function.
Then we seperatly fit a spline to the total $\sin(\delta)$ distribtuion and normalizes it over the declination range of the sample.
For a given time and declination the smooth function $R(t, \delta)$ gives the correct detector rate.

## Splinefit to sinDec

First we fit a spline to the sinDec distribtuion for all events.
This is equivalent to what is done in skylab to estimate the per signal background PDF from data.

In [ ]:
# This is equivalent to skylab's baclground PDF construction
sinDec_bins = 25
sinDec_range= [-1, 1]
sinDec_hist, sinDec_bins = np.histogram(exp["sinDec"], density=True,
                                        bins=sinDec_bins, range=sinDec_range)

m = get_binmids([sinDec_bins])[0]

if np.any(sinDec_hist <= 0.):
    raise ValueError(("Declination hist bins empty, this must not happen. "
                      +"Empty bin idx: {}".format(
                          np.arange(len(m))[sinDec_hist <= 0.])))

# Fit to logarithm, to avoid ringing. Raise err if evaluated outside range
sinDec_spline = sci.InterpolatedUnivariateSpline(m, np.log(sinDec_hist),
                                                 k=3, ext="extrapolate")

# Normalize to area on whole sky = 1, so norm = 2pi * integral(exp(spl))
def sinDec_pdf_(x):
    return np.exp(sinDec_spline(x))

norm = scint.quad(sinDec_pdf_, -1, 1)[0] * 2. * np.pi

def sinDec_pdf(x):
    return (np.exp(sinDec_spline(x))) / norm

print("SinDec pdf has area on 4pi = ", scint.quad(
    sinDec_pdf, -1, 1)[0] * 2 * np.pi)

In [ ]:
x = np.linspace(sinDec_bins[0], sinDec_bins[-1], 100)
y = sinDec_pdf(x)

plt.plot(x, y)

In [ ]:
# Try to hist dec dependence of single run
# As expected we have not enough statistic to see anything
run = 50
start = start_mjd[run]
stop = stop_mjd[run]
mask = (exp_times >= start) & (exp_times < stop)
_sinDec_run = np.sin(exp["dec"][mask])

plt.hist(_sinDec_run, range=[-1, 1], bins=10)

## Spline to rate distribution

Choosing spline weights according to scipy.interpolate.UnivariateSpline manual:

    If None (default), s = len(w) which should be a good value if 1/w[i] is an estimate of
    the standard deviation of y[i].

Internally it is doing a weighted least squares fit with $\sum_i(w_i(y_i-\text{spl}(x_i)))^2 \leq s$.
We leave $s$ as the default because we have an estimate for the stddevs: $\sigma_i = \sqrt{h_i}$.
To match the definition of the weights we use:

$$
    \frac{1}{w_i} \stackrel{!}{=} \sigma_i = \sqrt{h_i} \Leftrightarrow w_i 
                               = \frac{1}{\sqrt{h_i}}
$$

Because we scaled $h_i$ to get the rate in events per s, we need to scale the errors too:

$$
    \tilde{h}_i = \frac{h_i}{s_i} \Rightarrow \tilde{\sigma}_i = \frac{\sqrt{h_i}}{s_i} 
                = \frac{\sqrt{s_i\tilde{h}_i}}{s} 
                = \sqrt{\frac{\tilde{h}_i}{s_i}} = \frac{1}{w_i}
$$

The smoothing condition `s` chooses the support knots based on the weights.
Because we have some oscilating pattern due to to seasonal variations (periode ~1yr) a quadratic spline function is not enough.
So we choose the next higher order, a cubic spline, which is able to oscilate up and down exatcly once.

**Note:** If a weight is zero, the corresponding point doesn't contribute at all.
So we might consider using $w_i = \sigma_i$ instead.
Then point woth high poisson statsitics are preferred over low statistic bins.
It doesn't seem to make a huge difference though.

Below we try both weights and the unweighted case.
For the 'correctly' weighted case with $w_i = 1. / \sigma_i$ the spline oscillates strongly.
So we better try a true perdiodic function.

In [ ]:
# h is already scaled, so we need to scale the errors too
yerr = np.sqrt(h) / np.sqrt((stop_mjd - start_mjd) * secinday)
w = 1. / yerr
rate_spline = sci.UnivariateSpline(binmids, h, k=3, w=w,
                                   s=None, ext="extrapolate")

rate_spline_inv = sci.UnivariateSpline(binmids, h, k=3, w=1. / w,
                                       s=None, ext="extrapolate")

rate_spline_unw = sci.UnivariateSpline(binmids, h, k=3, w=None,
                                       s=None, ext="extrapolate")

In [ ]:
# Plot runs
xerr = 0.5 * (stop_mjd - start_mjd)
plt.errorbar(binmids, h, xerr=0, yerr=yerr, fmt=",")
plt.ylim(0, None);

# Plot spline
x = np.linspace(start_mjd[0], stop_mjd[-1], 200)
y = rate_spline(x)
plt.plot(x, y, zorder=5, lw=2, color="k", label="w=1/std")

# Plot weighted average. Weights are variance to resemble stddev weighted
# least squares fit
avg = np.average(h, weights=yerr**2)
plt.axhline(avg, 0, 1, color="k", ls="--", zorder=5)

# Plot unweighted mean and spline
y = rate_spline_unw(x)
plt.plot(x, y, zorder=5, lw=2, color="r")

avg = np.mean(h)
plt.axhline(avg, 0, 1, color="r", ls="--", zorder=5, label="w=1")

# Plot with inverse weights
y = rate_spline_inv(x)
plt.plot(x, y, zorder=5, lw=2, color="g", label="w=std")
avg = np.average(h, weights=1. / yerr**2)
plt.axhline(avg, 0, 1, color="g", ls="--", zorder=5)

plt.xlim(start_mjd[0], stop_mjd[-1])
plt.legend()
plt.tight_layout()
plt.savefig("./data/figs/time_rate_splines.png", dpi=200)

## Periodic function fit

Try a peridoc function with a weighted least squares fit.

$$
    f(x) = a\cdot \sin(b\cdot(x - c)) + d
$$

The least squares loss function is

$$
    R = \sum_i (w_i(y_i - f(x_i)))^2
$$

Weights are standard deviations from poisson histogram error.

$$
    w_i = \frac{1}{\sigma_i}
$$

Seed values are estimated from plot rate vs time.
Period should be 365 days (MJD) because we have one year of data so we choose $b0 = 2\pi/365$.
Amplitude is about $a_0=-0.0005$, because sinus seems to start with negative values.
The x-offset is choose as the first start date, to get the right order of magnitude.
The y-axis intersection $d$ schould be close to the weighted average, so we take this as a seed.

The bounds are motivated as follows (and if we don't hit them, it's OK to use them).
Amplitude $a$ should be positive, this also resolves a degenracy between a-axis offset.
The period $b$ should scatter around one year, a period larger than +-1 half a year is unphysical.
The x-offset $c$ cannot be greater than the initial +- the period because we have a periodic function.
The y-axis offset $d$ is arbitrarily constrained, but as seen from the plot it should not exceed 0.1. 

In [ ]:
def f(x, args):
    a, b, c, d = args
    return a * np.sin(b * (x - c)) + d

def lstsq(pars, *args):
    """
    Weighted leastsquares min sum((wi * (yi - fi))**2)
    """
    # data x,y-values and weights are fixed
    x, y, w = args[0], args[1], args[2]
    # Params get fitted
    a, b, c, d = pars[0], pars[1], pars[2], pars[3]
    # Target function
    f = a * np.sin(b * (x - c)) + d
    # Least squares loss
    return np.sum((w * (y - f))**2)

In [ ]:
# Seed values from consideration above.
a0 = -0.0005
b0 = 2. * np.pi / 365.
c0 = np.amin(start_mjd)
d0 = np.average(h, weights=yerr**2)

x0 = [a0, b0, c0, d0]
# Bounds as explained above
bounds = [[None, None], [0.5 * b0, 1.5 * b0], [c0 - b0, c0 + b0, ], [0, 0.01]]
# x, y values, weights
args = (binmids, h, 1. / yerr)

res = sco.minimize(fun=lstsq, x0=x0, args=args, bounds=bounds)

for i, name in enumerate(["Amplitude a", "Period b", "x-Shift c", "y-axis d"]):
    print(name, " : ", res.x[i])

In [ ]:
# Plot runs
xerr = 0.5 * (stop_mjd - start_mjd)
plt.errorbar(binmids, h, xerr=0, yerr=yerr, fmt=",")
plt.ylim(0, None);

# Plot fit
pars = res.x
x = np.linspace(start_mjd[0], stop_mjd[-1], 1000)
y = f(x, pars)
plt.plot(x, y, zorder=5)

# Plot y shift dashed to see baseline or years average
plt.axhline(res.x[3], 0, 1, color="C1", ls="--")

plt.xlim(start_mjd[0], stop_mjd[-1])
plt.tight_layout()

plt.savefig("./data/figs/time_rate_sinus.png", dpi=200)

## Combine both to make a time-dec rate function

Multiply the rate function of time with the pdf in sinDec.
This gives the rate per solid angle.
Integrated over the whole sphere, we recover the total rate that time.
Integrating further over the whole time range, regarding the deadtimes of the detector, we recover the number of total events in all runs in this sample.
We can approximate this by using the fitted y-axis offset, which is approximatly the mean and multiply with the livetitme.
We recover the number of total events to good approximation.

In [ ]:
print("Number of events from approx : ", res.x[3] * livetime * secinday)
print("True number of events        : ", len(exp))

# Simply integrating doesn't respect the downtimes
wrong = scint.quad(f, start_mjd[0], stop_mjd[-1], args=res.x)[0] * secinday
print("Integrating over whole year  : ", wrong)

In [ ]:
# Function of time, sinDec and right-ascension to get the rate at that point.
def time_sinDec_rate(sinDec, t):
    return sinDec_pdf(sinDec) * f(t, res.x)

In [ ]:
# This should yield ~1. The ratio of the fitted average d and the integral
# of the rate function over the whole sky at a time approximately at rate=d
_i = 2. * np.pi * scint.quad(time_sinDec_rate, -1, 1, args=55700)[0] / res.x[3]
print("1D and mukltiply by 2pi : ",_i)

# We can also use a 2D integrator to integrate RA as well (same result)
def fullsky_rate(ra, sinDec, t):
    return sinDec_pdf(sinDec) * f(t, res.x)
_i = scint.dblquad(fullsky_rate, -1, 1, lambda x: 0, lambda x: 2.*np.pi,
                   args=(55700,))[0] / res.x[3]
print("2D over dec and ra      : ", _i)

# Sampling the number of BG events to inject

Testing if the integration of the rate function gives the correct number of events to inject.
For small time windows it should be the same as just taking the rate times the window size (rectangular approximation).

In [ ]:
def rate_integral(trange, pars):
        """
        Match with factor [secinday] = 24 * 60 * 60 s / MJD = 86400/(Hz*MJD)
        in the last step.
            [a], [d] = Hz, [b], [c], [ti] = MJD
            [a / b] = Hz * MJD, [d * (t1 - t0)] = HZ * MJD
        """
        a, b, c, d = pars
        
        t0 = np.atleast_2d(trange[:, 0]).reshape(len(trange), 1)
        t1 = np.atleast_2d(trange[:, 1]).reshape(len(trange), 1)
        
        per = a / b * (np.cos(b * (t0 - c)) - np.cos(b * (t1 - c)))
        lin = d * (t1 - t0)

        return (per + lin) * secinday


def get_num_of_bg_events(t, trange, ntrials, pars):
    """
    Draw number of background events per trial from a poisson distribution
    with the mean of the fitted rate function.
    Then draw nevents times via rejection sampling for the time dpeendent rate
    function.
    
    Parameters
    ----------
    t : array-like
        Times of the occurance of each source event in MJD.
    trange : [float, float] or array_like, shape (len(t), 2)
        Time window(s) in seconds relativ to the given time(s) t.
        - If [float, float], the same window [lower, upper] is used for every
          source.
        - If array-like, lower [i, 0] and upper [i, 1] bounds of the time
          window per source.
    ntrials : int
        Number of background trials we need the number of how many events to
        inject for.
    pars : array-like
        Best fit parameters from the fit function used in its integral.
        
    Returns
    -------
    sample : list, length len(t)
        Contains a dict with keys "times" and "nevents"
    times : 
    nevents : array-like, shape (len(t), ntrials)
        The number of events to inject for each trial for each source.
    """
    t = np.array(t)
    trange = np.array(trange)
    nsrc = len(t)
    
    # Make shape (nsources, 1) for the times
    t = t.reshape(nsrc, 1)
    
    # If range is 1D (one for all) reshape it to (nsources, 2)
    if len(trange.shape) == 1:
        print("Using the same time window for all sources.")
        trange = np.repeat(trange.reshape(1, 2), repeats=nsrc, axis=0)
        
    # Prepare time window in MJD
    trange = t + trange / secinday
    
    # Expectation is the integral in the time frame
    expect = rate_integral(trange, pars)
        
    # Sample from poisson
    nevts = np.random.poisson(lam=expect, size=(nsrc, ntrials))
    
    return nevts

In [ ]:
t = start_mjd[100:104]
trange = np.array([-120, 220])
ntrials = 10

nevts = get_num_of_bg_events(t=t, trange=trange, ntrials=ntrials, pars=res.x)
print("Events to inject:\n", nevts)

# Test if integral and simple approximation is the same for small time frames
t = np.array(t)
trange = np.array(trange)
t = t.reshape(len(t), 1)
trange = np.repeat(trange.reshape(1, 2), repeats=len(t), axis=0)
trange = t + trange / secinday
rate = f(t, res.x)
nevts = rate_integral(trange, res.x)
print("\nFor small windows, rate * dt should be equal to integral")
print("Time window dt in seconds:\n", np.diff(trange))
print("Rate * dt:\n", rate * np.diff(trange, axis=1) * secinday)
print("Integral:\n", nevts)
print(np.allclose(rate * np.diff(trange, axis=1) * secinday, nevts))

# Time PDF ratio

Background in uniformly distributed in the time window.
Signal distribtution is falling off gaussian-like at both edges so normalization is different.
So the ratio S/B is simply the the signal pdf divided by the uniform normalization $1 / (t_1 - t_0)$ in the time frame.

To get finite support we truncate the gaussian edges at n sigma.
Though arbitrarliy introducet to smoothly run to zero, the concrete cutoff of the doesn't really matter (so say 4, 5, 6 sigma, etc).
This is because in the LLH we get the product of $\langle b_B \rangle B_i$.
A larger cutoff make the normalization of the BG pdf larger, but in the same time makes the number of expected BG event get higher in the same linear fashion.
So as long as we choose a cutoff which ensures that $S \approx 0$ outside, we're good to go.

In [ ]:
secinday = 24. * 60. * 60.

def time_bg_pdf(t, t0, a, b):
    """
    BG is uniform for t in [t0 + a, t0 + b] and 0 outside.
    
    Times t and t0 are given in MJD, the range is given relative to t0
    in seconds. t are the times we return pdf values for, t0 is the time of
    the source event around which the time frame is defined.
    
    The PDF is normed to time in seconds!
    """
    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
  
    pdf = np.zeros_like(_t, dtype=np.float)
    uni = (_t >= a) & (_t <= b)
    pdf[uni] = 1. / (b - a)
    return pdf

def time_sig_pdf(t, t0, dt, nsig=4):
    """
    Signal falls of with gaussian with sigma = dt outside uniform range dt.
    
    Times t, t0 are in MJD, dt is in seconds.
    t are the times we return pdf values for, t0 is the time of the source
    event around which the time frame is defined.
    dt is the time window starting from t0 in which signal is uniform.
    
    The PDF is normed to time in seconds!
    """
    if dt < 0:
        raise ValueError("dt must not be negative.")

    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
    
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    sig_t = np.clip(dt, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling and zero
    gr = (_t < 0) & (_t >= -sig_t_clip)
    gf = (_t > dt) & (_t <= dt + sig_t_clip)
    uni = (_t >= 0) & (_t <= dt)
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=0, scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt, scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize whole distribtuion
    dcdf = (scs.norm.cdf(dt + sig_t_clip, loc=dt, scale=sig_t) -
            scs.norm.cdf(-sig_t_clip, loc=0., scale=sig_t))
    norm = dcdf + dt / gaus_norm
    
    return pdf / norm

def time_soverb(t, t0, dt, nsig):
    """
    Time signal over background PDF.
    
    Signal and background PDFs are each normalized over seconds.
    Signal PDF has gaussian edges to smoothly let it fall of to zero, the
    stddev is dt when dt is in [2, 30]s, otherwise the nearest edge.

    To ensure finite support, the edges are truncated after nsig * dt.

    Parameters
    ----------
    t : array-like
        Times given in MJD for which we want to evaluate the ratio.
    t0 : float
        Time of the source event.
    dt : float
        Time window in seconds starting from t0 in which the signal pdf is
        assumed to be uniform. Must not be negative.
    nsig : float
        Clip the gaussian edges at nsig * dt
    """
    if dt < 0:
        raise ValueError("dt must not be negative.")

    secinday = 24. * 60. * 60.

    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
   
    # Create signal PDF
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    sig_t = np.clip(dt, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling
    gr = (_t < 0) & (_t >= -sig_t_clip)
    gf = (_t > dt) & (_t <= dt + sig_t_clip)
    uni = (_t >= 0) & (_t <= dt)
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=0, scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt, scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize signal distribtuion
    dcdf = (scs.norm.cdf(dt + sig_t_clip, loc=dt, scale=sig_t) -
            scs.norm.cdf(-sig_t_clip, loc=0., scale=sig_t))
    norm = dcdf + dt / gaus_norm
    pdf /= norm
    
    # Calculate the ratio
    bg_pdf = 1. / (dt + 2 * sig_t_clip)
    ratio = pdf / bg_pdf
    return ratio

In [ ]:
# Plot a the signal and BG PDFs for a single case
# Arbitrary start date from data
t0 = start_mjd[100]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = 10
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
plt_rng = [-clip, dt + clip]
t = np.linspace(t0_sec + plt_rng[0], t0_sec + plt_rng[1], 200) / secinday

bg_pdf = time_bg_pdf(t, t0, -clip, dt + clip)
sig_pdf = time_sig_pdf(t, t0, dt, nsig)

# Plot in normalized time
_t = t * secinday - t0 * secinday
plt.plot(_t, bg_pdf, "C0-")
plt.plot(_t, sig_pdf, "C1-")
plt.axvline(dt, 0, 1, color="C7", ls="--")
plt.axvline(0, 0, 1, color="C1", ls="--")

plt.xlabel("Time relative to t0 in sec")
plt.ylim(0, None);
plt.show()

# Integrate both pdf over time range to show they are correctly normalized
# Note that PDFs are defined in second so we multiply by secinday 
bg_int = scint.quad(time_bg_pdf, t[0], t[-1],
                    args=(t0, -clip, dt + clip))[0] * secinday
sig_int = scint.quad(time_sig_pdf, t[0], t[-1],
                    args=(t0, dt, nsig))[0] * secinday

print("BG integral     : ", bg_int)
print("Signal integral : ", sig_int)

In [ ]:
# Make a plot with ratios for different time windows as in the paper
# Arbitrary start date from data
t0 = start_mjd[100]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dts = [5, 50, 200]
nsig = 4

# Make t values for plotting in MJD around t0, fitting all in one plot
max_dt = np.amax(dts)
clip = np.clip(max_dt, 2, 30) * nsig
plt_rng = np.array([-clip, max_dt + clip])
t = np.linspace(t0_sec + 1.2 *plt_rng[0],
                t0_sec + 1.2 * plt_rng[1], 1000) / secinday
_t = t * secinday - t0 * secinday

# Mark event time
plt.axvline(0, 0, 1, c="#353132", ls="--", lw=2)

colors = ["C0", "C3", "C2"]
for i, dt in enumerate(dts):
    # Plot ratio S/B
    SoB = time_soverb(t, t0, dt, nsig)
    plt.plot(_t, SoB, lw=2, c=colors[i],
             label=r"$T_\mathrm{{uni}}$: {:>3d}s".format(dt))
    # Fill uniform part, might look nicely
    # fbtw = (_t > 0) & (_t < dt)
    # plt.fill_between(_t[fbtw], 0, SoB[fbtw], color="C7", alpha=0.1)

# Make it look like the paper plot, but with slightly extended borders, to
# nothing breaks outside the total time frame
plt.xlim(1.2 * plt_rng)
plt.ylim(0, 3)
plt.xlabel("t - t0 in sec")
plt.ylabel("S / B")
plt.legend(loc="upper right")
plt.grid()
plt.show()

# Declination bump in data a south

At declination -pi/2 there are a lot of events, that show up as a large spike in sin(dec).
Where does this come from?
Those events come directly from above (southern sky, zenith=0)

In [ ]:
bins = 20
ev_dec = exp["dec"]
sin_dec = np.sin(ev_dec)

fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 4))
_ = axl.hist(sin_dec, bins=200, normed=False)
_ = axr.hist(np.rad2deg(ev_dec), bins=500, normed=False)

axr.set_xlabel("sinDec")
axr.set_xlabel("Dec in °")

axl.arrow(x=-1, y=1000, dx=0, dy=-100, head_length=20, head_width=0.03, lw=2,
          fc="C1", ec="C1")
axr.arrow(x=-90, y=300, dx=0, dy=-200, head_length=20, head_width=3, lw=2,
          fc="C1", ec="C1")

fig.tight_layout()

# Integrate Kent function in 2D

The paraboloid sigma was gained from estimating a circular region, in which the 2D llh includes 39% of the total probability (Attention: chi2(1, df=2), because 2D gaussian behave differently).
This assumes a gaussian llh function, so when using the usual gaussian in the signal pdf we can directly use this sigma estimate for the gaussian sigma.

For a Kent distribtuions kappa however, it is unclear how both are connected.
For starters, when kappa is 0, points are uniform on the sky, but fully concentrated in a gaussian and vice versa.
We could simply assume $\kappa = 1/\sigma$ but this is mostly founded on the 1D Mises distribution which has this property for small sigma only.

So here we try the following:

- For a fixed sigma, try to find the corresponding kappa, so that when integrating the Kent PDF with this kappa in a circle with radius sigma, the probability content in that circle is 68%.

The Kent PDF is given by:

$$
    f(\vec{x}_i|\vec{x}_S) = \frac{\kappa}{4\pi\sinh{\kappa}}\cdot\exp(\kappa(\vec{x}_i\cdot\vec{x}_S))
                           = \frac{\kappa}{4\pi\sinh{\kappa}}\cdot\exp(\kappa\cos\Psi)
$$

where $\cos\Psi$ is the angular distance of both vectors.

Because we need to integrate over the surface of the unit sphere, we define the Kent PDF in terms of spherical coordinates as used in equatorial coordinates.

$$
    \vec{x} = \begin{pmatrix}
                \cos\theta\cos{\varphi} \\ \cos\theta\sin{\varphi} \\ \sin\theta
              \end{pmatrix}
$$

In the integration we must not forget to multiply with the functional determinant (surface element) $\mathrm{d}A=-r^2\cos\varphi\mathrm{d}\theta\mathrm{d}\varphi$ for the convention used here.

Without loss of generality we can assume that the mean vector (=src position) is pointing along the z-axis $(0,0,1)$.
This way we have easy integration boundaries, the radius of the circle is then directly given by the polar angle $\theta$:

$$
    \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix} \cdot
      \begin{pmatrix} \cos\theta\cos{\varphi} \\ \cos\theta\sin{\varphi} \\ \sin\theta \end{pmatrix}
    = \sin\theta
$$

so our full integral assembles to:

\begin{align}
    \mathrm{Prob\ Content\ within\ }\sigma = \alpha &= \int_{\theta=\pi/2}^{\pi/2 - \sigma}\int_{\varphi=0}^{2\pi}
        \frac{\kappa}{4\pi\sinh{\kappa}}\cdot\exp(\kappa\sin\theta)\cdot(-\cos\theta)
        \mathrm{d}\theta\mathrm{d}\varphi \\
      &= 2\pi\cdot\frac{\kappa}{4\pi\sinh{\kappa}}\cdot \int_{\theta=\pi/2}^{\pi/2 - \sigma}
        -\cos\theta\exp(\kappa\sin\theta)\mathrm{d}\theta\mathrm{d}\varphi \\
      &= \frac{\kappa}{2\sinh{\kappa}}\cdot \int_{\theta=\pi/2 - \sigma}^{\pi/2}
        \cos\theta\exp(\kappa\sin\theta)\mathrm{d}\theta\mathrm{d}\varphi
\end{align}

We can solve this analytially:

\begin{align}
    \alpha &= \frac{1}{2\sinh{\kappa}}\cdot \int_{\theta=\pi/2 - \sigma}^{\pi/2}
               \kappa\cos\theta\exp(\kappa\sin\theta)\mathrm{d}\theta
            = \frac{1}{2\sinh{\kappa}}\cdot \left[\exp(\kappa\sin\theta)\right]_{\theta=\pi/2 - \sigma}^{\pi/2} \\
            &= \frac{1}{2\sinh{\kappa}}\cdot \left[\exp(\kappa) - \exp(\kappa\cos(\sigma))\right]
\end{align}

This automatically reduces to $\sinh\kappa / \sinh\kappa = 1$ for $\sigma=180°$ and to zero for $\sigma=0°$ as expected.

The special case $\kappa=0$ resembles a uniform distribtuion on the sky and is given by:

\begin{align}
    \alpha &= \int_{\theta=\pi/2 - \sigma}^{\pi/2}\int_{\varphi=0}^{2\pi}
               \frac{\cos\theta}{4\pi} \mathrm{d}\theta\mathrm{d}\varphi
           =\frac{1}{2}\int_{\theta=\pi/2 - \sigma}^{\pi/2}\cos\theta\mathrm{d}\theta \\
           &= \frac{\sin(\pi/2) - \sin(\pi/2-\sigma)}{2}
           = \frac{1 - \cos\sigma}{2}
\end{align}

Below we run some tests up to which events sigma we can go with the Kent distribution.
Also we see, if it is justified to identify $\kappa = 1/\sigma**2$.

In [ ]:
# Quickly see that the pdf itself is normalized.
# Note: Integral is in cos(theta), so no area element needed.
# For very large kappas this breaks because the stepsize is too coarse
def S(cosDist, kappa):
    return (kappa / (2. * np.pi * (1. - np.exp(-2. * kappa))) *
                 np.exp(kappa * (cosDist - 1. )))

sig = np.deg2rad(2)
kappa = 1 / sig**2
scint.quad(S, a=3*sig, b=1, args=(kappa))[0] * 2 * np.pi

In [ ]:
# First let's get the 1 sigma gaussian probability content to compare against
one_sig = scs.norm.cdf(1) - scs.norm.cdf(-1)

def alpha_in_kent_stable(kappa, sigma):
    """
    moxe is a numeric god :+1:
    """
    sigma = np.atleast_1d(sigma)
    if np.any(sigma > np.pi):
        raise ValueError("Angular error greater than pi.")
    if kappa == 0:
        return 0.5 * (1. - np.cos(sigma))

    a = 1. / np.tanh(kappa) + 1.
    b = 0.5 * (1. - np.exp(kappa * (np.cos(sigma) - 1.)))
    return a * b

def alpha_in_kent(kappa, sigma):
    sigma = np.atleast_1d(sigma)
    if np.any(sigma > np.pi):
        raise ValueError("Angular error greater than pi.")
    if kappa == 0:
        return 0.5 * (1. - np.cos(sigma))
    
    return 0.5 * (np.exp(kappa) - np.exp(kappa * np.cos(sigma))) / np.sinh(kappa)

def kappa_from_sigma(sigma, alpha=0.39):
    # Wrapper because we need the point alpha(k) - alpha = 0
    def fun(kappa):
        return alpha_in_kent_stable(kappa, sigma) - alpha
    
    x0 = float(1. / sigma**2)
    return sco.newton(fun, x0=x0)

In [ ]:
# Up to ~40° kappa = 1/sigma**2 is OK to use. Then the norm kicks in
sigmas = np.linspace(0.1, 150, 1000)
alpha = scs.chi2.cdf(1**2, df=2)

kappas = np.zeros_like(sigmas)
for i, sig in enumerate(sigmas):
    kappas[i] = kappa_from_sigma(np.deg2rad(sig), alpha=alpha)
    
# Simple subtitution as in mrichmans thesis
subst = 1. / np.deg2rad(sigmas)**2

plt.plot(sigmas, kappas, label=r"Exact")
plt.plot(sigmas, subst, label=r"$\kappa = 1/\sigma^2$")

plt.xlabel("sigma in °")
plt.ylabel(r"$\kappa$")
plt.yscale("log")
plt.legend()

# plt.savefig("data/figs/kent_kappa.png", dpi=200)

In [ ]:
# Different view of the plot above, now in terms of different kappas
sigmas = np.linspace(0, 20, 360 + 1)
sigmas_rad = np.deg2rad(sigmas)
kappas = np.arange(10, 200, 20)

alpha = scs.chi2.cdf(1**2, df=2)  # ca. 39%, 2D gaussian
plt.axhline(1, 0, 1, color=dg, ls="--")
plt.axhline(alpha, 0, 1, color=dg, ls="--")
for k in kappas:
    a = alpha_in_kent(k, sigmas_rad)
    plt.plot(sigmas, a, label=r"$\kappa = {:.1f}$".format(k))

# Note: if evt sigma is larger than ~80°, we can't describe the correct
#       probability content with a Kent distribution. An artifact from the
#       LLH beeing not gaussian enough.
plt.xlabel("Evt sigma in °")
plt.ylabel("Prob. in sigma")
plt.xlim(0, 20)
plt.ylim(0, 1.05)
plt.legend()
plt.title("CDF of symmetric Kent distribution")

# plt.savefig("data/figs/kent_cdf.png", dpi=200)

In [ ]:
# Now directly compare Kent to 2D gaussian (provided by mrichman :+1:)
# For sigmas near the turnover ~75°, we can see, that the Kent line crosses
# the gaussian. It smoothes out more to get exactly the 39% at the sigma level
mth = 90
theta = np.linspace (0, np.deg2rad(mth), 1000)
sig_deg = [3, 5, 10, 20, 30, 75]

cmap = plt.cm.get_cmap("plasma")

for sd in sig_deg:
    sr = np.radians(sd)
    # kappa = 1 / sr**2
    kappa = kappa_from_sigma(sigma=sr, alpha=scs.chi2.cdf(1**2, df=2))
    G = np.exp(-theta**2 / (2 * sr**2)) / (2. * np.pi * sr**2)
    K = kappa / (4. * np.pi * np.sinh(kappa)) * np.exp(kappa * np.cos(theta))
    color = cmap((sd - min(sig_deg)) / max(sig_deg))
    plt.plot(np.rad2deg(theta), K, color=color,
             label=r'$\sigma={}^\circ$'.format (sd))
    plt.plot(np.rad2deg(theta), G, ls='--', color=color)

plt.yscale("log")
plt.xlabel (r'$\Delta\Psi~[^\circ]$')
plt.ylabel (r'PDF')
plt.xlim (0, mth)
plt.ylim (1e-3, 1e3)
plt.legend (loc='best', ncol=2, prop={"size": 'small'}, title="Kent")
plt.tight_layout()

# plt.savefig("data/figs/kent_vs_gaus.png", dpi=200)

# Tests for the KDE integration

## Marginalize KDE by integration

Instead of sampling and reducing to 2D histograms, we can try to truly integrate one dimension of the KDE to be able to plot also the tails of the distribution, where events usually end up only in large samples.

In [ ]:
# KDE CV is running on cluster and pickles the GridSearchCV
fname = "./data/kde_cv/KDE_model_selector_20_exp_IC86_I_followup_2nd_pass.pickle"
with open(fname, "rb") as f:
    model_selector = pickle.load(f)

kde = model_selector.best_estimator_
bw = model_selector.best_params_["bandwidth"]
print("Best bandwidth : {:.3f}".format(bw))

# We maybe just want to stick with the slightly overfitting kernel to
# be as close as possible to data
OVERFIT = True
if OVERFIT:
    bw = 0.075
    kde = skn.KernelDensity(bandwidth=bw, kernel="gaussian", rtol=1e-8)
print("Used bandwidth : {:.3f}".format(bw))

# KDE sample must be cut in sigma before fitting, similar to range in hist
_exp = exp[exp["sigma"] <= np.deg2rad(5)]

fac_logE = 1.5
fac_dec = 2.5
fac_sigma = 2.

_logE = fac_logE * _exp["logE"]
_sigma = fac_sigma * np.rad2deg(_exp["sigma"])
_dec = fac_dec * _exp["dec"]

kde_sample = np.vstack((_logE, _dec, _sigma)).T

# Fit KDE best model to sample
kde.fit(kde_sample)

# Make some samples
nsamples_kde = int(1e7)
bg_samples = kde.sample(n_samples=nsamples_kde)

### 1D case

Integrate out 2 axis with a double integral to show a 1D margin distribution.
This take super long, 5 Minutes per point.
But it can be parallelized pretty simple if needed.
Code to create the values is on phobos.

**Resumee:** Sampling many values and simply bin in 1D is much better, needs less time and is probably more accurate.

In [ ]:
# Compare to data hist
_ = plt.hist(exp["logE"] * fac_logE, bins=100, normed=True, label="data")

# Compare 'true' integration with
h, b = np.histogram(bg_samples[:, 0], bins=200, range=[2, 10], normed=True)
m = 0.5 * (b[:-1] + b[1:])
_ = plt.plot(m, h, label="sample")

bins_and_vals = np.load("data/2d_integrate_kde/bins_and_vals.npy")
x = bins_and_vals[0]
vals = bins_and_vals[1]
_ = plt.plot(x, vals, label="integrated")

plt.legend()
plt.savefig("./kde.png", dpi=200)

### 2D PDF

Integrate out only one axis (here sigma) to show the 2D marginalized PDF.
Executed on phobos, takes a little while on 50x50 grid.

In [ ]:
# Get precalculated integral data
bins = np.load("data/1d_integrate_kde/logE_sinDec_bins_50x50.npy")
vals = np.load("data/1d_integrate_kde/logE_sinDec_int_50x50.npy")

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Compare to data hist
_ = ax[0].hist2d(exp["logE"], np.sin(exp["dec"]), bins=bins, normed=True)

# Compare 'true' integration
mids = get_binmids(bins)
xx, yy  = map(np.ravel, np.meshgrid(mids[0], mids[1]))
_ = ax[1].hist2d(xx, yy, bins=bins, weights=vals)

fig.tight_layout()

## Justify the sigma cut

Only few higher energy events from the sothern sky are excluded (see cut=10).
But really bad reconstructed events tend to have higher energies (see cut=90).
Still it should be OK to remove those > 10 because they have not so much spatial information.

In [ ]:
# Show the leftover event s after a sigma cut
sig_cut = 10
m = exp["sigma"] > np.deg2rad(sig_cut)

_ = plt.hist2d(exp["logE"][m], np.rad2deg(exp["dec"][m]),
               bins=30, cmap="inferno")
plt.colorbar()
plt.title("Total Evts w sigma > {:d}°: {:d} ({:.3f}%)".format(
        sig_cut, np.sum(m), np.sum(m) / len(exp) * 100))
plt.xlabel("logE")
plt.ylabel("dec in °")
plt.show()

# Show the skewed sigma distribution with the cut applied and mean vs median

## Test the marginalize_hist method

It should be equivalent to use one of the following methods to create a 1D histogram from the original 3D data pdf in logE, dec and sigma:

1. Simply use the original 1D data in any variable, e.g. simply histogram logE
2. Create the complete 3D histogram and marginalize by summing over remaining dimensions.

When using unnormalized hists, 2. is simply summing up all other counts.

When using normalized hists, we need to sum with respect to the binwidths in the current dimension to keep the normalization intact.
This is only useful, when only the histogram is available and not the original sample.

We want to compare if both methods are equivalent
As we can see, all ratios are one, so methods are equal.

In [ ]:
def make_hist_ratio(h1, h2):
    """Return the ratio h1 / h2. Return 0 where h2 is 0."""
    m = (h2 > 0)
    ratio = np.zeros_like(h1)
    ratio[m] = h1[m] / h2[m]
    return ratio

### Unnormalized
First the unnormalized version. Simply sum over the other axes of the 3D hist.

In [ ]:
# Plot each variable in a single plot and the ratios seperately
fig, [[axtl, axtr], [axbl, axbr]] = plt.subplots(2, 2, figsize=(10, 8))

# We also make a cut < 10° in sigma, because there are some outliers
m = exp["sigma"] <= np.deg2rad(10)
sigma = np.rad2deg(exp["sigma"][m])
logE = exp["logE"][m]
dec = np.sin(exp["dec"][m])

logE_nbins = 50
dec_nbins = 40
sigma_nbins = 30

# Make the 3D hist
sample = np.vstack((logE, dec, sigma)).T
nbins = [logE_nbins, dec_nbins, sigma_nbins]
h, b = np.histogramdd(sample, bins=nbins,)

# Get binmids for plotting
m = get_binmids(b)

# Common hist settings
h1 = {"lw": 2, "color": "k", "histtype": "step"}
h2 = {"lw": 2, "color": "r", "histtype": "step", "alpha": 0.5}

# logE
logE_h, logE_b, _ = axtl.hist(logE, bins=logE_nbins, **h1)
logE_hm = np.sum(h, axis=(1, 2))
_ = axtl.hist(m[0], bins=b[0], weights=logE_hm, **h2)
# Ratio plot below
axtl_sec = split_axis(axtl, "bottom", "20%", cbar=False)
axtl_sec.hist(m[0], b[0], weights=make_hist_ratio(logE_h, logE_hm), **h2)
axtl_sec.axhline(1, 0, 1, color="k")
axtl_sec.set_ylim(0, 2)

# dec
dec_h, dec_b, _ = axbl.hist(dec, bins=dec_nbins, **h1)
dec_hm = np.sum(h, axis=(0, 2))
_ = axbl.hist(m[1], bins=b[1], weights=dec_hm, **h2)

axbl_sec = split_axis(axbl, "bottom", "20%", cbar=None)
axbl_sec.hist(m[1], b[1], weights=make_hist_ratio(dec_h, dec_hm), **h2)
axbl_sec.axhline(1, 0, 1, color="k")
axbl_sec.set_ylim(0, 2)

# sigma
sigma_h, sigma_b, _ = axtr.hist(sigma, bins=sigma_nbins, **h1)
sigma_hm = np.sum(h, axis=(0, 1))
_ = axtr.hist(m[2], bins=b[2], weights=sigma_hm, **h2)

axtr_sec = split_axis(axtr, "bottom", "20%", cbar=None)
axtr_sec.hist(m[2], b[2], weights=make_hist_ratio(sigma_h, sigma_hm), **h2)
axtr_sec.axhline(1, 0, 1, color="k")
axtr_sec.set_ylim(0, 2)

axbr.set_visible(False)

fig.suptitle("Black: 1D, Red: Margin", fontsize=15);

### Normalized
Sum over the other axes of the 3D hist and multiply by bin widths.

In [ ]:
# Plot each variable in a single plot and the ratios seperately
fig, [[axtl, axtr], [axbl, axbr]] = plt.subplots(2, 2, figsize=(10, 8))

# Now make it normed
h, b = np.histogramdd(sample, bins=nbins, normed=True)

# Get binmids for plotting
m = get_binmids(b)

# logE
logE_h, logE_b, _ = axtl.hist(logE, bins=logE_nbins, normed=True, **h1)
logE_hm = hist_marginalize(h=h, bins=b, axes=(1, 2))[0]
_ = axtl.hist(m[0], bins=b[0], weights=logE_hm, **h2)
# Ratio plot below
axtl_sec = split_axis(axtl, "bottom", "20%", cbar=False)
axtl_sec.hist(m[0], b[0], weights=make_hist_ratio(logE_h, logE_hm), **h2)
axtl_sec.axhline(1, 0, 1, color="k")
axtl_sec.set_ylim(0, 2)

# dec
dec_h, dec_b, _ = axbl.hist(dec, bins=dec_nbins, normed=True, **h1)
dec_hm = hist_marginalize(h=h, bins=b, axes=(0, 2))[0]
_ = axbl.hist(m[1], bins=b[1], weights=dec_hm, **h2)

axbl_sec = split_axis(axbl, "bottom", "20%", cbar=None)
axbl_sec.hist(m[1], b[1], weights=make_hist_ratio(dec_h, dec_hm), **h2)
axbl_sec.axhline(1, 0, 1, color="k")
axbl_sec.set_ylim(0, 2)

# sigma
sigma_h, sigma_b, _ = axtr.hist(sigma, bins=sigma_nbins, normed=True, **h1)
sigma_hm = hist_marginalize(h=h, bins=b, axes=(0, 1))[0]
_ = axtr.hist(m[2], bins=b[2], weights=sigma_hm, **h2)

axtr_sec = split_axis(axtr, "bottom", "20%", cbar=None)
axtr_sec.hist(m[2], b[2], weights=make_hist_ratio(sigma_h, sigma_hm), **h2)
axtr_sec.axhline(1, 0, 1, color="k")
axtr_sec.set_ylim(0, 2)

axbr.set_visible(False)

fig.suptitle("Black: 1D, Red: Margin", fontsize=15);

# Mask sinDec logE ratio PDF spline

Assign each high E events for which no data but MC is present the maximum ratio from signal over background and all lowE the lowest ratio.

In skylab all events which no data but MC events get the highest value, which is strange at the lowE regime.

In [ ]:
# Prepare the MC data, signal weighted to astro unbroken power law
gamma = 2.
# No flux norm, because we normalize anyway
mc_w = mc["ow"] * mc["trueE"]**(-gamma)

# Make the MC hist. Use this binning for the data too
mc_sindec = np.sin(mc["dec"])
mc_logE = mc["logE"]
bins = [50, 50]
range = [[-1, 1], [1, 10]]
mc_h, bx, by = np.histogram2d(mc_sindec, mc_logE, bins=bins, range=range,
                              weights=mc_w, normed=True)

# Make the data hist
b = [bx, by]
bg_logE = exp["logE"]
bg_sindec = np.sin(exp["dec"])
bg_h, _, _ = np.histogram2d(bg_sindec, bg_logE, bins=b,
                                range=range, normed=True)


# 4 cases:
#   - Data & MC: Calculate the ratio
#   - (>logEthresh) & (no data or no MC): Assign highest normal ratio
#   - (<logEthresh) & (no data or no MC): Assign lowest normal ratio
#   - No data and no MC): Assign any value (eg 1), these are never accessed
# Get logE value per bin in entrie histogram
m = get_binmids(b)
logEs =  np.repeat(m[1], repeats=bins[1]).reshape(bins).T
logEthresh = 3.5

m1 = (bg_h > 0) & (mc_h > 0)
m2 = (logEs > logEthresh) & ((bg_h <= 0) | (mc_h <= 0))
m3 = (logEs <= logEthresh) & ((bg_h <= 0) | (mc_h <= 0))
m4 = (bg_h <= 0) & (mc_h <= 0)

SoB = np.ones_like(bg_h)
SoB[m1] = mc_h[m1] / bg_h[m1]
SoB[m2] = np.amax(SoB)
SoB[m3] = np.amin(SoB)
SoB[m4] = 1.

# Make a 4 color mask map
mask_map = np.ones_like(bg_h)
mask_map[m1] = 1.
mask_map[m2] = 2.
mask_map[m3] = 0.
mask_map[m4] = -1.

# Plot it
fig, ((axtl, axtr), (axbl, axbr)) = plt.subplots(2, 2, figsize=(12, 10))
xx, yy = map(np.ravel, np.meshgrid(*m))

_ = axtl.hist2d(xx, yy, bins=b, weights=bg_h.T.flatten(), cmap="viridis",
               norm=LogNorm())
_ = axtr.hist2d(xx, yy, bins=b, weights=mc_h.T.flatten(), cmap="viridis",
               norm=LogNorm())

# Plot mask
cbins = np.linspace(-1, 2, 5)
cticks = 0.5 * (cbins[:-1] + cbins[1:])
cmap = matplotlib.cm.get_cmap("inferno", len(cticks))
_, _, _, img = axbr.hist2d(xx, yy, bins=b, weights=mask_map.T.flatten(),
                          cmap=cmap)
cax = split_axis(axbr, cbar=True)
cbar = plt.colorbar(cax=cax, mappable=img, ticks=cticks)
cbar.ax.set_yticklabels(["No BG AND no MC", "lowE, no D OR MC", "BG AND MC",
                         "higE, no D OR MC"],
                        rotation=60, va="center")

# Plot ratio
cn = max(np.amin(SoB), np.amax(SoB))
_, _, _, img = axbl.hist2d(xx, yy, bins=b, weights=SoB.T.flatten(),
                          cmap="coolwarm", vmin=1./cn, vmax=cn, norm=LogNorm())
cax = split_axis(axbl, cbar=True)
cbar = plt.colorbar(cax=cax, mappable=img)

fig.tight_layout()
plt.savefig("./data/figs/energy_ratio_mask.png", dpi=200)
plt.show()

# Compare sigma to x*exp(-x)

In [ ]:
# Data
# sig_deg = np.rad2deg(exp["sigma"][exp["sigma"] < np.deg2rad(10)])
# _, b, _ = plt.hist(sig_deg, bins=50, normed=True)

# PDF
x = np.linspace(0, 10, 500)
a = 3
y = a**2 * x * np.exp(-a * x)
_ = plt.plot(x, y, lw=2)

# Sampled from PDF
# Combo from Pythia 8 and Trial&Error. How to derive from LambertW function?
#   http://home.thep.lu.se/~torbjorn/doxygen/Basics_8h_source.html
u1, u2 = np.random.uniform(size=(2, 10000))
sam = -np.log(u1 * u2) / a
plt.hist(sam, bins=b, normed=True, histtype="step", lw=2)

plt.xlim(0, 5)

# Old Scaled KDE

To save the old non-adaptive and kind-of guessed KDE for comparison, we put the former code in here.

## 3D histogram of BG data
First we make a 3D histogram to better compare to mrichmann and to get an overview over the distribution.

In [ ]:
# HANDTUNED scale parameter to "fit" KDE expectation to data...
# TODO: Use Adaptive kernel width and asymmetric gaus kernels
#       For sigma it might make sense to a take a restricted kernel [0, inf]
fac_logE = 1.5
fac_dec = 2.5
fac_sigma = 2.

logE = fac_logE * exp["logE"]
sigma = fac_sigma * np.rad2deg(exp["sigma"])
# np.cos(np.pi / 2. + exp["dec"]); dec is for {sin(dec), dec, cos(zen)}
dec = fac_dec * exp["dec"]

# Binning is rather arbitrary because we don't calc stuff with the hist
bins = [50, 50, 50]
# Range for sigma is picked by looking at the 1D distribution and cutting of
# the tail. This will be covered by the KDE tail anyway. Rest is default
r = [[np.amin(logE), np.amax(logE)],
     [np.amin(dec), np.amax(dec)],
     [0., fac_sigma * 5.]]

sample = np.vstack((logE, dec, sigma)).T
h, bins = np.histogramdd(sample=sample, bins=bins, range=r, normed=False)

# Make bin mids for later use
mids = []
for b in bins:
    mids.append(0.5 * (b[:-1] + b[1:]))

# Make a nice corner plot
fig, ax = corner_hist(h, bins=bins,
                      label=["scaled logE", "scaled dec", "scaled sigma deg"],
                      hist2D_args={"cmap": "Greys"},
                      hist_args={"color":"#353132"})

# plt.savefig("./data/figs/bg_corner_scaled.png", dpi=200)

## Kernel Density Estimation

We use scikit learn's cross validation with a gaussian kernel to get the most robust bandwidth.
Then we integrate with the same binning as above and compare to the 3D histogram.

This section relies heavily on [Jake van der Plas examples for KDE](https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/).
More info on how KDE cross validation works can be found in [Modern Nonparametric Methods](http://www2.stat.duke.edu/~wjang/teaching/S05-293/lecture/ch6.pdf).

In [ ]:
# KDE CV is running on cluster and pickles the GridSearchCV
fname = "./data/kde_cv/KDE_model_selector_20_exp_IC86_I_followup_2nd_pass.pickle"
with open(fname, "rb") as f:
    model_selector = pickle.load(f)

kde = model_selector.best_estimator_
bw = model_selector.best_params_["bandwidth"]
print("Best bandwidth : {:.3f}".format(bw))

# We maybe just want to stick with the slightly overfitting kernel to
# be as close as possible to data
OVERFIT = True
if OVERFIT:
    bw = 0.075
    kde = skn.KernelDensity(bandwidth=bw, kernel="gaussian", rtol=1e-8)
print("Used bandwidth : {:.3f}".format(bw))

# KDE sample must be cut in sigma before fitting, similar to range in hist
_exp = exp[exp["sigma"] <= np.deg2rad(5)]

_logE = fac_logE * _exp["logE"]
_sigma = fac_sigma * np.rad2deg(_exp["sigma"])
_dec = fac_dec * _exp["dec"]

kde_sample = np.vstack((_logE, _dec, _sigma)).T

# Fit KDE best model to sample
kde.fit(kde_sample)

# Generate some BG samples to compare to the original data hist.
# Use more statistics, histograms get normalized and we want the best estimate
# for the pdf
nsamples_kde = int(1e7)
bg_samples = kde.sample(n_samples=nsamples_kde)

# Make histogram with same binning as original data
bg_h, bg_bins = np.histogramdd(sample=bg_samples, bins=bins, range=r, normed=True)

fig, ax = corner_hist(bg_h, bins=bg_bins,
                      label=["scaled logE", "scaled sin(dec)",
                             "scaled sigma deg"],
                      hist2D_args={"cmap": "Greys"},
                      hist_args={"color":"#353132"})

# plt.savefig("./data/figs/kde_corner_scaled.png", dpi=200)

## Compare KDE to original data

Make a ratio histogram of the KDE sample and the original data sample.

### 2D marginalization

In [ ]:
# Create 2D hists, by leaving out one parameter
xlabel = ["scaled " + s for s in ["logE", "logE", "dec"]]
ylabel = ["scaled " + s for s in ["dec", "sigma in °", "sigma in °"]]

for i, axes in enumerate([[0, 1], [0, 2], [1, 2]]):
    _b = np.array(bins)
    h_exp, b_exp = np.histogramdd(sample[:, axes],
                                  bins=_b[axes], normed=True)
    h_kde, b_kde = np.histogramdd(bg_samples[:, axes],
                                  bins=_b[axes], normed=True)
    
    # KDE is expectation, but sampled with much more events.
    # Weights would simply scale the total number of KDE events to match the
    # number of original events. That would be the mean for the poisson
    # distribution in each bin. So to get OK KDE expectation sqrt(n) errors
    # in each bin, we divide not by the number of drawn KDE but by the number
    # of original events.   
    # Again shapes of meshgrid and hist are transposed
    diffXX, _ = np.meshgrid(np.diff(_b[0]), np.diff(_b[1]))
    norm_kde = len(exp) * diffXX.T
    sigma_kde = np.sqrt(h_kde / norm_kde)

    # Make 3 different diff/ratio hists to estimate KDE quality in
    # 1D marginalization.
    m = (h_exp > 0.)
    ratio_h = np.zeros_like(h_exp)
    ratio_h[m] = h_kde[m] / h_exp[m]

    diff_h = h_kde - h_exp

    m = (sigma_kde > 0.)
    sigma_ratio_h = np.zeros_like(h_exp)
    sigma_ratio_h[m] = (h_exp[m] - h_kde[m]) / sigma_kde[m]

    # Bin mids and hist grid
    _b = b_exp
    m = get_binmids(_b)
    xx, yy = map(np.ravel, np.meshgrid(m[0], m[1]))
    
    
    # Big plot on the left and three right
    fig = plt.figure(figsize=(10, 6))
    gs = gridspec.GridSpec(3, 3)
    axl = fig.add_subplot(gs[:, :2])
    axrt = fig.add_subplot(gs[0, 2])
    axrc = fig.add_subplot(gs[1, 2])
    axrb = fig.add_subplot(gs[2, 2])
    
    # Steal space for colorbars
    caxl = split_axis(axl, "right")
    caxrt = split_axis(axrt, "left")
    caxrc = split_axis(axrc, "left")
    caxrb = split_axis(axrb, "left")

    # Unset top and center xticklabels as they are shared with the bottom plot
    axrt.set_xticklabels([])
    axrc.set_xticklabels([])
        
    # Left: Difference over KDE sigma
    # cbar_extr = max(np.amax(sigma_ratio_h),  # Center colormap to min/max
    #                         abs(np.amin(sigma_ratio_h)))
    _, _, _, imgl = axl.hist2d(xx, yy, bins=_b, weights=sigma_ratio_h.T.ravel(),
                               cmap="seismic", vmax=5, vmin=-5)
    cbarl = plt.colorbar(cax=caxl, mappable=imgl)
    axl.set_xlabel(xlabel[i])
    axl.set_ylabel(ylabel[i])
    axl.set_title("(exp - kde) / sigma_kde")
    
    # Right top: Ratio
    _, _, _, imgrt = axrt.hist2d(xx, yy, bins=_b, weights=ratio_h.T.ravel(),
                                 cmap="seismic", vmax=2, vmin=0);
    cbarrt = plt.colorbar(cax=caxrt, mappable=imgrt)
    axrt.set_title("kde / exp")

    # Right center: Data hist
    _, _, _, imgrc = axrc.hist2d(xx, yy, bins=_b, weights=h_exp.T.ravel(),
                                 cmap="Greys", norm=LogNorm());
    cbarrc = plt.colorbar(cax=caxrc, mappable=imgrc)
    axrc.set_title("exp logscale")

    # Right bottom: KDE hist, same colorbar scale as on data
    _, _, _, imgrb = axrb.hist2d(xx, yy, bins=_b, weights=h_kde.T.ravel(),
                                 cmap="Greys", norm=LogNorm());
    # Set with same colormap as on data
    imgrb.set_clim(cbarrc.get_clim())
    cbarrb = plt.colorbar(cax=caxrb, mappable=imgrb)
    axrb.set_title("kde logscale")
    
    # Set tick and label positions
    for ax in [caxrt, caxrc, caxrb]:
        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_left()
    
    fig.tight_layout()
    plt.savefig("./data/figs/kde_data_2d_{}_{}.png".format(
                    xlabel[i], ylabel[i]),
                dpi=200)
    plt.show()

### 1D marginalization

In [ ]:
# Pseudo smooth marginalization is done by sampling many point from KDE an
# using a finely binned 1D histogram, so it looks smooth
xlabel = ["scaled " + s for s in ["logE", "dec", "sigma °"]]

for i, axes in enumerate([0, 1, 2]):
    _b = np.array(bins)
    h_exp, b_exp = np.histogram(sample[:, axes],
                                bins=_b[axes], normed=True)
    h_kde, b_kde = np.histogram(bg_samples[:, axes],
                                bins=_b[axes], normed=True)
    
#     h_exp, b_exp = hist_marginalize(h, bins, axes=axes)
#     h_kde, b_kde = hist_marginalize(bg_h, bg_bins, axes=axes)
      
    # KDE errorbars as in 2D case
    norm_kde = len(exp) * np.diff(b_kde)
    sigma_kde = np.sqrt(h_kde / norm_kde)

    # Make 3 different diff/ratio hists to estimate KDE quality in
    # 1D marginalization.
    m = (h_exp > 0.)
    ratio_h = np.zeros_like(h_exp)
    ratio_h[m] = h_kde[m] / h_exp[m]

    diff_h = h_kde - h_exp

    m = (sigma_kde > 0.)
    sigma_ratio_h = np.zeros_like(h_exp)
    sigma_ratio_h[m] = (h_exp[m] - h_kde[m]) / sigma_kde[m]

    # Bin mids
    _b = b_exp
    m = get_binmids([_b])[0]
    
    # Plot both and the ration normed. Big plot on the left and three right
    fig = plt.figure(figsize=(10, 6))
    gs = gridspec.GridSpec(3, 3)
    axl = fig.add_subplot(gs[:, :2])
    axrt = fig.add_subplot(gs[0, 2])
    axrc = fig.add_subplot(gs[1, 2])
    axrb = fig.add_subplot(gs[2, 2])

    axrt.set_xticklabels([])
    axrc.set_xticklabels([])

    # Set ticks and labels right
    for ax in [axrt, axrc, axrb]:
        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_right()

    # Limits
    for ax in [axl, axrt, axrc, axrb]:
        ax.set_xlim(_b[0], _b[-1])
        
    # Main plot:
    # Plot more dense to mimic a smooth curve
    __h, __b = np.histogram(bg_samples[:, i], bins=500,
                            range=[_b[0], _b[-1]], density=True)
    __m = get_binmids([__b])[0]
    axl.plot(__m, __h, lw=3, alpha=0.5)
    
    _ = axl.hist(m, bins=_b, weights=h_exp, label="exp", histtype="step",
                 lw=2, color="k")
    _ = axl.errorbar(m, h_kde, yerr=sigma_kde, fmt=",", color="r")
    _ = axl.hist(m, bins=_b, weights=h_kde, label="kde", histtype="step",
                 lw=2, color="r")    
    
    axl.set_xlabel(xlabel[i])
    axl.legend(loc="upper right")

    # Top right: Difference
    _ = axrt.axhline(0, 0, 1, color="k", ls="-")
    _ = axrt.hlines([-.02, -.01, .01, .02], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrt.hist(m, bins=_b, weights=diff_h, histtype="step", lw=2, color="r")
    axrt.set_ylim(-.05, +.05)
    axrt.set_ylabel("kde - exp")

    # Center right: Ratio
    _ = axrc.axhline(1, 0, 1, color="k", ls="-")
    _ = axrc.hlines([0.8, 0.9, 1.1, 1.2], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrc.hist(m, bins=_b, weights=ratio_h, histtype="step", lw=2, color="r")
    axrc.set_ylim(.5, 1.5)
    axrc.set_ylabel("kde / exp")

    # Bottom right: Ratio of diff to sigma of expectation
    _ = axrb.axhline(0, 0, 1, color="k", ls="-")
    _ = axrb.hlines([-2, -1, 1, 2], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrb.hist(m, bins=_b, weights=sigma_ratio_h, histtype="step", lw=2, color="r")
    axrb.set_ylim(-3, +3)
    axrb.set_ylabel("(exp-kde)/sigma_kde")
    
    plt.savefig("./data/figs/kde_data_1d_{}.png".format(
            xlabel[i]), dpi=200)
    plt.show()

# Show scores of awKDE CV

In [ ]:
with open("data/awKDE_CV/CV10_glob_bw_alpha_EXP_IC86I_CUT_sig.ll.20_PARS_" +
          "diag_True_pass2.pickle", "rb") as f:
    model_selector = pickle.load(f)
    
glob_bws = model_selector.cv_results_["param_glob_bw"].compressed()
mean_score = model_selector.cv_results_["mean_test_score"]
std_score = model_selector.cv_results_["std_test_score"]

# Mark best
_ = plt.axvline(glob_bws[bf_idx], 0, 1, ls="--", lw=2, color="C2", zorder=-1)
# Each horizontal line is one alpha tested for that glob_bw
bf_idx = np.argmax(mean_score)
_ = plt.errorbar(glob_bws, mean_score, yerr=std_score,fmt="_")

plt.xlabel("global bandwidth")

# Old BG Rate Sampler

Tried to sample everything at once (trials, src, times).
Very clumsy because for each src and trial, a different number of evt is sampled, so we can't use arrays.

Also we can fit a single LLH at a time only, so no need to get all samples in advance.
Might be faster, but consumes also much more memory.

## Sample number of events in frame

In [ ]:
def filter_runs(run):
    """
    Filter runs as stated in jfeintzig's doc.
    """
    exclude_runs = [120028, 120029, 120030, 120087, 120156, 120157]
    if ((run["good_i3"] == True) & (run["good_it"] == True) &
        (run["run"] not in exclude_runs)):
        return True
    else:
        return False
    
goodrun_dict, _livetime = hlp.create_goodrun_dict(
    runlist="data/runlists/ic86-i-goodrunlist.json", filter_runs=filter_runs)

h = hlp._create_runtime_bins(exp["timeMJD"], goodrun_dict=goodrun_dict,
                             remove_zero_runs=True)

def f(x, pars):
    """
    Returns the rate at a given time in MJD.
    """
    a, b, c, d = pars
    return a * np.sin(b * (x - c)) + d

def lstsq(pars, *args):
    """
    Weighted leastsquares min sum((wi * (yi - fi))**2)
    """
    # data x,y-values and weights are fixed
    x, y, w = args
    _f = f(x, pars)
    return np.sum((w * (y - _f))**2)

# Seed values from consideration above.
# a0 = -0.0005
# b0 = 2. * np.pi / 365.  # We could restrict the period to one yr exact.
# c0 = np.amin(start_mjd)
# d0 = np.average(h, weights=yerr**2)

rate = h["rate"]
rate_std = h["rate_std"]
X = exp["timeMJD"]
binmids = 0.5 * (h["start_mjd"] + h["stop_mjd"])

a0 = 0.5 * (np.amax(rate) - np.amin(rate))
b0 = 2. * np.pi / 365.
c0 = np.amin(X)
d0 = np.average(rate, weights=rate_std**2)

x0 = [a0, b0, c0, d0]
# Bounds as explained above
bounds = [[None, None], [0.5 * b0, 1.5 * b0], [c0 - b0, c0 + b0, ], [0, 0.01]]
# x, y values, weights
args = (binmids, rate, 1. / rate_std)

res = sco.minimize(fun=lstsq, x0=x0, args=args, bounds=bounds)
bf_pars = res.x

print("Amplitude   : {: 13.5f} in Hz".format(res.x[0]))
print("Period (d)  : {: 13.5f} in days".format(2 * np.pi / res.x[1]))
print("Offset (MJD): {: 13.5f} in MJD".format(res.x[2]))
print("Avg. rate   : {: 13.5f} in Hz".format(res.x[3]))

# Define the rate function:
def rate_fun(t):
    """
    Returns the rate at a given time in MJD.
    
    Parameters
    ----------
    t : array-like
        Time in MJD.
        
    Returns
    -------
    rate : array-like
        The rate of background events in Hz.
    """
    return f(t, res.x)

In [ ]:
secinday = 24 * 60 * 60
def _prep_times(t, trange):
    """
    Little wrapper to not DRY.
    """
    t = np.atleast_1d(t)
    trange = np.array(trange)
    nsrc = len(t)
    
    # Make shape (nsources, 1) for the times
    t = t.reshape(nsrc, 1)
    
    # If range is 1D (one for all) reshape it to (nsources, 2)
    if len(trange.shape) == 1:
        trange = np.repeat(trange.reshape(1, 2), repeats=nsrc, axis=0)
        
    # Prepare time window in MJD
    trange = t + trange / secinday
    
    return t, trange

def get_num_of_bg_events(t, trange, ntrials, pars):
    """
    Draw number of background events per trial from a poisson distribution
    with the mean of the fitted rate function.
    Then draw nevents times via rejection sampling for the time dpeendent rate
    function.
    
    Parameters
    ----------
    t : array-like
        Times of the occurance of each source event in MJD.
    trange : [float, float] or array_like, shape (len(t), 2)
        Time window(s) in seconds relativ to the given time(s) t.
        - If [float, float], the same window [lower, upper] is used for every
          source.
        - If array-like, lower [i, 0] and upper [i, 1] bounds of the time
          window per source.
    ntrials : int
        Number of background trials we need the number of how many events to
        inject for.
    pars : array-like
        Best fit parameters from the fit function used in its integral.
        
    Returns
    -------
    nevents : array-like, shape (len(t), ntrials)
        The number of events to inject for each trial for each source.
    """
    # Integrate rate function analytially in desired interval
    def rate_integral(trange, pars):
        """
        Match with factor [secinday] = 24 * 60 * 60 s / MJD = 86400/(Hz*MJD)
        in the last step.
            [a], [d] = Hz, [b], [c], [ti] = MJD
            [a / b] = Hz * MJD, [d * (t1 - t0)] = HZ * MJD
        """
        a, b, c, d = pars
        
        t0 = np.atleast_2d(trange[:, 0]).reshape(len(trange), 1)
        t1 = np.atleast_2d(trange[:, 1]).reshape(len(trange), 1)
        
        per = a / b * (np.cos(b * (t0 - c)) - np.cos(b * (t1 - c)))
        lin = d * (t1 - t0)

        return (per + lin) * secinday
    
    t, trange = _prep_times(t, trange)
    
    # Expectation is the integral in the time frame
    expect = rate_integral(trange, pars)
        
    # Sample from poisson
    nevts = np.random.poisson(lam=expect, size=(len(t), ntrials))
      
    return nevts

In [ ]:
start, stop = h["start_mjd"], h["stop_mjd"]

nsrc = 5
t = start[100:100 + nsrc]
# Make different time windows to verify that more events are sampled in more time
trange = np.array([[-10 * 5*i, 20 * 5*i] for i in range(1, nsrc + 1)])
print("Time windows:\n", trange)

ntrials = 10
nevts = get_num_of_bg_events(t, trange, ntrials, res.x)
# print("All evts, per src and trial:\n", nevts)

# Total evts in all trials per src
print("Total sampled evts per src:\n", np.sum(nevts, axis=1)[:, np.newaxis])

# Compare to expectation
print("Expected per trial per src:\n", np.diff(trange, axis=1) * res.x[3])
print("Sampled per trial per src:\n", np.sum(nevts, axis=1)[:, np.newaxis] / ntrials)

## Now the sampling of random times in the time frame

First we want to see, that all BG injected events stay in the correct time frame and make a uniform distribution for small time frames.

Then we make the time window really big and the events should follow the rate function PDF.

In [ ]:
def get_times_in_frame(t, trange, nsamples):
    """
    Parameters
    ----------
    t : float
        Time of the occurance of the source event in MJD.
    trange : [float, float]
        Time window in seconds relativ to the given time t.
    nsamples : array-like, type int, shape (len(t))
        Number of events to inject per trial. Number of trials is given by
        the length of nsamples.
        
    Returns
    -------
    times : list, length len(t)
        List of samples times in MJD of background events per source.
        For each source i nsamples[i] times are drawn from the rate function.
    """
    _pdf = rate_fun
    
    t, trange = _prep_times(t, trange)
    
    sample = []
    nsamples = np.atleast_1d(nsamples)
    
    for i, ni in enumerate(nsamples):
        sam, _ = rejection_sampling(_pdf, bounds=trange, n=ni)
        sample.append(sam)
        
    return sample

In [ ]:
# First the small time frame
# Arbitrary start date from data
t0 = start[100]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = 200
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
plt_rng = [-clip, dt + clip]
trange = plt_rng
ntrials = 10000

# Sample times
nevts = get_num_of_bg_events(t=t0, trange=trange, ntrials=ntrials,
                             pars=res.x)[0]
times = get_times_in_frame(t0, trange, nevts)

# Plot them in together with the PDFs
def time_bg_pdf(t, t0, a, b):
    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
  
    pdf = np.zeros_like(_t, dtype=np.float)
    uni = (_t >= a) & (_t <= b)
    pdf[uni] = 1. / (b - a)
    return pdf

def time_sig_pdf(t, t0, dt, nsig=4):
    if dt < 0:
        raise ValueError("dt must not be negative.")

    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
    
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    sig_t = np.clip(dt, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling and zero
    gr = (_t < 0) & (_t >= -sig_t_clip)
    gf = (_t > dt) & (_t <= dt + sig_t_clip)
    uni = (_t >= 0) & (_t <= dt)
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=0, scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt, scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize whole distribtuion
    dcdf = (scs.norm.cdf(dt + sig_t_clip, loc=dt, scale=sig_t) -
            scs.norm.cdf(-sig_t_clip, loc=0., scale=sig_t))
    norm = dcdf + dt / gaus_norm
    
    return pdf / norm


# Plot the pdfs
t = np.linspace(t0_sec + plt_rng[0], t0_sec + plt_rng[1], 200) / secinday
bg_pdf = time_bg_pdf(t, t0, -clip, dt + clip)
sig_pdf = time_sig_pdf(t, t0, dt, nsig)

# Plot in normalized time
_t = t * secinday - t0 * secinday
plt.plot(_t, bg_pdf, "C0-")
plt.plot(_t, sig_pdf, "C1-")
plt.axvline(dt, 0, 1, color="C7", ls="--")
plt.axvline(0, 0, 1, color="C1", ls="--")

# Plot injected events from all trials
T = np.array([])
for ti in times:
    T = np.append(T, ti)  
T = (T - t0) * secinday

_ = plt.hist(T, bins=50, normed=True, color=dg, alpha=.25)

plt.xlabel("Time relative to t0 in sec")
plt.ylim(0, None);
plt.tight_layout()

# plt.savefig("./data/figs/bg_events_time_sampled_narrow.png", dpi=200)

plt.show()

In [ ]:
# Now the really large time frame, over the whole time range
t0 = start[0]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = (stop[-1] - start[0]) * secinday
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
plt_rng = [-clip, dt + clip]
trange = plt_rng
ntrials = 1

# Sample times
nevts = get_num_of_bg_events(t=t0, trange=trange, ntrials=ntrials,
                             pars=res.x)[0]
times = get_times_in_frame(t0, trange, nevts)

# Plot injected events from all trials
T = np.array([])
for ti in times:
    T = np.append(T, ti)  

h, b = np.histogram(T, bins=1081)
m = get_binmids([b])[0]
scale = np.diff(b) * secinday * ntrials
yerr = np.sqrt(h) / scale
h = h / scale

plt.errorbar(m, h, yerr=yerr, fmt=",")

# Plot normalized rate function to compare
t = np.linspace(start[0], stop[-1], 100)
r = rate_fun(t=t)
plt.plot(t, r, lw=2, zorder=5)
plt.axhline(res.x[3], 0, 1, color="C1", ls="--", label="", zorder=5)

plt.xlim(start[0], stop[-1])
plt.ylim(0, 0.009)
plt.tight_layout()

# plt.savefig("./data/figs/bg_events_time_sampled_wide.png", dpi=200)

plt.show()

# Test Production Modules

Test if the modules work correctly.
They contain the same code as in the main test notebook, but can be used as classes.
This should simplyfy production.

Currently each submodul only does a very special task:

- `bg_injector`: Samples ("injects") backgorund events for trials
- `bg_rate_injector`: Samples ("injects") the number of BG events to be injected per trial.
- `rate_function`: Describes the time depence of the background rate.
- `llh`: Implements the likelihood function and signal and background PDFs.
- `signal_injector`: Same as `bg_injector` but injecting signal evts from MC.
- `analysis`: Main module pulling it all together, making trials, fitting llhs, provides methods for advanced tasks.

In [ ]:
_exp = exp[exp["sigma"] < np.deg2rad(20)]
_logE = _exp["logE"]
_dec = _exp["dec"]
_sigma = _exp["sigma"]

sample = np.vstack((_logE, _dec, _sigma)).T

## BG Injector

Injects information for background-like events.

In [ ]:
n_samples = 100000

### Data Resampling

In [ ]:
data_inj = BGInj.DataBGInjector()
data_inj.fit(sample)
data_sam = data_inj.sample(n_samples)

xlabel = ["logE", "sinDec", "logE"]
ylabel = ["sinDec", "sigma", "sigma"]
for i, axes in enumerate([[0, 1], [1, 2], [0,2]]):
    fig, (al, ar) = hlp.hist_comp(sample[:, axes], data_sam[:, axes])
    al.set_xlabel(xlabel[i])
    ar.set_xlabel(xlabel[i])
    al.set_ylabel(ylabel[i])
    ar.set_ylabel(ylabel[i])
    al.set_title("Data")
    ar.set_title("Data sample: {} evts from original Data".format(len(data_sam)))
    plt.show()

### Adaptive Width KDE sampling

In [ ]:
# Assign model from CV, which has already evaluated adaptive kernels
with open("data/awKDE_CV/CV10_glob_bw_alpha_EXP_IC86I_CUT_sig.ll.20_" +
          "PARS_diag_True_pass2.pickle", "rb") as f:
    model_selector = pickle.load(f)
    print(model_selector.best_params_)

kde_inj = BGInj.KDEBGInjector()
kde_inj.kde_model = model_selector.best_estimator_

# We could still change the alpha, but the global bandwidth must stay fixed
# kde_inj.kde_model.alpha = 0.3

# Fit doesn't take long because all adaptive kernels are set
bounds = np.array([[None, None], [-np.pi / 2. , np.pi / 2.], [0, None]])
kde_inj.fit(sample, bounds)

# Sample with bounds, because KDEs spillover
bounds = np.array([[None, None], [-np.pi / 2. , np.pi / 2.], [0, None]])
kde_sam = kde_inj.sample(n_samples)

xlabel = ["logE", "sinDec", "logE"]
ylabel = ["sinDec", "sigma", "sigma"]
for i, axes in enumerate([[0, 1], [1, 2], [0,2]]):
    fig, (al, ar) = hlp.hist_comp(sample[:, axes], kde_sam[:, axes])
    al.set_xlabel(xlabel[i])
    ar.set_xlabel(xlabel[i])
    al.set_ylabel(ylabel[i])
    ar.set_ylabel(ylabel[i])
    al.set_title("Data")
    ar.set_title("KDE sample: {} evts".format(len(kde_sam)))
    plt.show()

### GRBLLH style

In [ ]:
# If False, only sample where data was
# If True sample in global min/max bounding box
minmax = True

mrinj = BGInj.MRichmanBGInjector()
ax0_bins, ax1_bins, ax2_bins = mrinj.fit(sample, nbins=10, minmax=minmax)
mr_sam = mrinj.sample(n_samples=n_samples)

xlabel = ["logE", "sinDec", "logE"]
ylabel = ["sinDec", "sigma", "sigma"]
for i, axes in enumerate([[0, 1], [1, 2], [0,2]]):
    fig, (al, ar) = hlp.hist_comp(sample[:, axes], mr_sam[:, axes])
    al.set_xlabel(xlabel[i])
    ar.set_xlabel(xlabel[i])
    al.set_ylabel(ylabel[i])
    ar.set_ylabel(ylabel[i])
    al.set_title("Data")
    ar.set_title("Pseudo MR sample: {} evts".format(len(mr_sam)))
    plt.show()

### Pseudo Data (uniform) sampling

In [ ]:
uni_inj = BGInj.UniformBGInjector()
uni_sam = uni_inj.sample(n_samples)

for axes in [[0, 1], [1, 2], [0,2]]:
    fig, (al, ar) = hlp.hist_comp(sample[:, axes], uni_sam[:, axes])
    al.set_title("Data")
    ar.set_title("Pseudo (uniform) sample: {} evts".format(len(uni_sam)))
    plt.show()

## BG Rate Injector

This module injects times of background like events.

### Injector created from runlist

Fit sinus to rates from detector runs

In [ ]:
# First create a rate function
rate_func = RateFunc.Sinus1yrRateFunction()

In [ ]:
# Now parse the rundict and make the fitted injector from that
def filter_runs(run):
    """
    Filter runs as stated in jfeintzig's doc.
    """
    exclude_runs = [120028, 120029, 120030, 120087, 120156, 120157]
    if ((run["good_i3"] == True) & (run["good_it"] == True) &
        (run["run"] not in exclude_runs)):
        return True
    else:
        return False
    
# Let's create an injector using a goodrun list. This creates a run dict
runlist="data/runlists/ic86-i-goodrunlist.json"
runlist_inj = BGRateInj.RunlistBGRateInjector(runlist, filter_runs, rate_func)

# Fit function to exp times to runlist bins
times = exp["timeMJD"]
rate_func = runlist_inj.fit(T=times, x0=None, remove_zero_runs=True)

In [ ]:
# Plot runs
rates = runlist_inj.rate_rec
start_mjd = rates["start_mjd"]
stop_mjd = rates["stop_mjd"]

xerr = 0.5 * (stop_mjd - start_mjd)
yerr = rates["rate_std"]
binmids = 0.5 * (stop_mjd + start_mjd)

plt.errorbar(binmids, rates["rate"], xerr=xerr, yerr=yerr, fmt=",")
plt.ylim(0, None);

# Plot fit
t = np.linspace(start_mjd[0], stop_mjd[-1], 1000)
y = rate_func(t)
plt.plot(t, y, zorder=5)

# Plot y shift dashed to see baseline or years average
avg = runlist_inj.best_pars[2]
plt.axhline(avg, 0, 1, color="C1", ls="--", label="")

plt.xlim(start_mjd[0], stop_mjd[-1])
plt.xlabel("MJD")
plt.ylabel("Rate in Hz")

# plt.savefig("./data/figs/time_rate_sinus.png", dpi=200)
plt.ylim(0, 0.009)
plt.tight_layout()
plt.show()

print("Best fit params:\n", runlist_inj.best_pars)

Sample many trials at once for a single src and time

In [ ]:
rates = runlist_inj.rate_rec
start_mjd = rates["start_mjd"]

# Pick some random time and time frame
t = np.random.choice(start_mjd, size=1)
trange = np.array([-120, 220])

ntrials = int(1e6)
trials = runlist_inj.sample(t, trange, ntrials=ntrials)

nevents = np.array(list(map(len, trials)))
print("Sampled total of {:d} events in {:d} trials.".format(
        np.sum(nevents), ntrials))

# Plot poisson distribution of nevents with expectation from integral
expect = runlist_inj.best_estimator_integral(t, trange)
_ = plt.hist(nevents, bins=np.arange(10), normed=True)
plt.axvline(expect, 0, 1, color="C1", ls="--", lw=2)
x = np.arange(0, 10)
y = scs.poisson.pmf(x, mu=expect)
_ = plt.plot(x, y, "C1", lw=2, drawstyle="steps-post")

In [ ]:
# First the small time frame
# Arbitrary start date from data
t0 = np.random.choice(start_mjd, size=1)
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = 200
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
plt_rng = np.array([-clip, dt + clip])
trange = plt_rng
ntrials = 10000

# Sample times
trials = runlist_inj.sample(t0, trange, ntrials=ntrials)

# Plot them in together with the PDFs
def time_bg_pdf(t, t0, a, b):
    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
  
    pdf = np.zeros_like(_t, dtype=np.float)
    uni = (_t >= a) & (_t <= b)
    pdf[uni] = 1. / (b - a)
    return pdf

def time_sig_pdf(t, t0, dt, nsig=4):
    if dt < 0:
        raise ValueError("dt must not be negative.")

    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
    
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    sig_t = np.clip(dt, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling and zero
    gr = (_t < 0) & (_t >= -sig_t_clip)
    gf = (_t > dt) & (_t <= dt + sig_t_clip)
    uni = (_t >= 0) & (_t <= dt)
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=0, scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt, scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize whole distribtuion
    dcdf = (scs.norm.cdf(dt + sig_t_clip, loc=dt, scale=sig_t) -
            scs.norm.cdf(-sig_t_clip, loc=0., scale=sig_t))
    norm = dcdf + dt / gaus_norm
    
    return pdf / norm


# Plot the pdfs
t = np.linspace(t0_sec + plt_rng[0], t0_sec + plt_rng[1], 200) / secinday
bg_pdf = time_bg_pdf(t, t0, -clip, dt + clip)
sig_pdf = time_sig_pdf(t, t0, dt, nsig)

# Plot in normalized time
_t = t * secinday - t0 * secinday
plt.plot(_t, bg_pdf, "C0-")
plt.plot(_t, sig_pdf, "C1-")
plt.axvline(dt, 0, 1, color="C3", ls="--")
plt.axvline(0, 0, 1, color="C2", ls="--")

# Plot injected events from all trials, relative times
times = [i for arr in trials for i in arr] # Flatten out to single array
times = (times - t0) * secinday
_ = plt.hist(times, bins=50, normed=True, color=dg, alpha=.25)

plt.xlabel("Time relative to t0 in sec")
plt.ylim(0, None);
plt.tight_layout()

# plt.savefig("./data/figs/bg_events_time_sampled_narrow.png", dpi=200)

plt.show()

In [ ]:
# Now the really large time frame, over the whole time range
t0 = start_mjd[0]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = (stop_mjd[-1] - start_mjd[0]) * secinday
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
plt_rng = [-clip, dt + clip]
trange = plt_rng
ntrials = 1

# Sample times
trials = runlist_inj.sample(t0, trange, ntrials=ntrials)
times = [i for arr in trials for i in arr] # Flatten out to single array

h, b = np.histogram(times, bins=1081)
m = get_binmids([b])[0]
scale = np.diff(b) * secinday * ntrials
yerr = np.sqrt(h) / scale
h = h / scale

plt.errorbar(m, h, yerr=yerr, fmt=",")

# Plot normalized rate function to compare
t = np.linspace(start_mjd[0], stop_mjd[-1], 100)
r = runlist_inj.best_estimator(t)
plt.plot(t, r, lw=2, zorder=5)
plt.axhline(runlist_inj.best_pars[2], 0, 1, color="C1",
            ls="--", label="", zorder=5)

plt.xlim(start_mjd[0], stop_mjd[-1])
plt.ylim(0, 0.009)
plt.tight_layout()

# plt.savefig("./data/figs/bg_events_time_sampled_wide.png", dpi=200)

plt.show()

## BG Rate Function

Test if fit, sample and integral works, with a simple example.

### SinusRateFunction

In [ ]:
# Define parameters for the test function
period_days = 300.
b = 2 * np.pi / period_days  # Period in 1/MJD
c = 0  # t-Offset in MJD
d = 1  # Rate offset in Hz = 1 evt/sec is average -> 86400 evts/day
a = d / 2.  # Amplitude in Hz = +- 0.5 evts / per second
pars = np.array([a, b, c, d])

sinfun = RateFunc.SinusRateFunction()

# Plot function
t0, t1 = c, c + period_days
t = np.linspace(0, t1, 200)  # In MJD days
y = sinfun.fun(t, pars)

_ = plt.plot(t, y, lw=2, label="fun")

# Plot integral
intgrl = np.zeros_like(t)
for i, ti in enumerate(t):
    intgrl[i] = sinfun.integral(t=t0, trange=[t0, ti*secinday], pars=pars)
    
# Scale integral, we expect 24*3600=86400 evts/day * (period_days days)
print("Expect   : ", secinday * t1)
print("Integral : ", intgrl[-1])
_ = plt.plot(t, intgrl / 1e7, lw=2, label="integral/1e7")

# Sample from whole range and scale normed hist with time scale to match rate
nsam = int(1e4)
sam = sinfun.sample(t=t0, trange=[t0, t1*secinday], pars=pars, n_samples=nsam)
h, b = np.histogram(sam, range=[t0, t1], bins=50, density=True)
m = get_binmids([b])[0]
_ = plt.hist(m, bins=b, weights=h * (t1 - t0), color="C0",
             alpha=0.5, label="sampled")

# Finally fit the sampled points again
runtime = (t1 - t0)
p0 = None  # Test default args
bf_pars = sinfun.fit(t=m, rate=h * (t1 - t0), rate_std=None, p0=p0)
yfit = sinfun.fun(t, bf_pars)
_ = plt.plot(t, yfit, lw=2, color="C3", ls="--", label="fitted")
p0 = sinfun._get_default_seed(t=m, rate=h * (t1 - t0),
                              rate_std=np.ones_like(m))
yseed = sinfun.fun(t, p0)
_ = plt.plot(t, yseed, lw=2, color="C3", ls="-", alpha=0.3,
             label="default seed")

plt.xlabel("time in MJD")
plt.ylabel("rate in Hz")
_ = plt.ylim(0, None)
plt.legend()
plt.tight_layout()

# plt.savefig("data/figs/rate_function_test.png", dpi=200)

plt.show()

### Sinus1yrRateFunction

The same as above, but now with fixed period of 1 year.

In [ ]:
# Define parameters for the test function
period_days = 365.25
c = 0  # t-Offset in MJD
d = 1  # Rate offset in Hz = 1 evt/sec is average -> 86400 evts/day
a = d / 2.  # Amplitude in Hz = +- 0.5 evts / per second
pars = np.array([a, c, d])

sinfun = RateFunc.Sinus1yrRateFunction()

# Plot function
t0, t1 = c, c + period_days
t = np.linspace(0, t1, 200)  # In MJD days
y = sinfun.fun(t, pars)

_ = plt.plot(t, y, lw=2, label="fun")

# Plot integral
intgrl = np.zeros_like(t)
for i, ti in enumerate(t):
    intgrl[i] = sinfun.integral(t=t0, trange=[t0, ti*secinday], pars=pars)
    
# Scale integral, we expect 24*3600=86400 evts/day * (period_days days)
print("Expect   : ", secinday * t1)
print("Integral : ", intgrl[-1])
_ = plt.plot(t, intgrl / 1e7, lw=2, label="integral/1e7")

# Sample from whole range and scale normed hist with time scale to match rate
nsam = int(1e4)
sam = sinfun.sample(t=t0, trange=[t0, t1*secinday], pars=pars, n_samples=nsam)
h, b = np.histogram(sam, range=[t0, t1], bins=50, density=True)
m = get_binmids([b])[0]
_ = plt.hist(m, bins=b, weights=h * (t1 - t0), color="C0",
             alpha=0.5, label="sampled")

# Finally fit the sampled points again
runtime = (t1 - t0)
p0 = None  # Test default args
bf_pars = sinfun.fit(t=m, rate=h * (t1 - t0), rate_std=None, p0=p0)
yfit = sinfun.fun(t, bf_pars)
_ = plt.plot(t, yfit, lw=2, color="C3", ls="--", label="fitted")
p0 = sinfun._get_default_seed(t=m, rate=h * (t1 - t0),
                              rate_std=np.ones_like(m))
yseed = sinfun.fun(t, p0)
_ = plt.plot(t, yseed, lw=2, color="C3", ls="-", alpha=0.3,
             label="default seed")

plt.xlabel("time in MJD")
plt.ylabel("rate in Hz")
_ = plt.ylim(0, None)
plt.legend()
plt.tight_layout()

# plt.savefig("data/figs/rate_function_test.png", dpi=200)

plt.show()

### ConstantRateFunction

Use constant rate function but leave the sinus to see how the fit behaves.
Otherwise it would be boring to just see 3 flat lines over another.

In [ ]:
# Define sinus parameters
period_days = 365.25
c = 0  # t-Offset in MJD
d = 1  # Rate offset in Hz = 1 evt/sec is average -> 86400 evts/day
a = d / 2.  # Amplitude in Hz = +- 0.5 evts / per second
sinpars = np.array([a, c, d])

# Same for the constant function.
constpars = (d,)

sinfun = RateFunc.Sinus1yrRateFunction()
constfun = RateFunc.ConstantRateFunction()

# Plot sinus and constant function
t0, t1 = c, c + period_days
t = np.linspace(0, t1, 200)  # In MJD days
y = sinfun.fun(t, sinpars)
yc = constfun.fun(t, constpars)

_ = plt.plot(t, y, color="C0", lw=2, ls="--")
_ = plt.plot(t, yc, lw=2, label="fun")

# Plot integral
intgrl = np.zeros_like(t)
for i, ti in enumerate(t):
    intgrl[i] = constfun.integral(t=t0, trange=[t0, ti*secinday],
                                  pars=constpars)
    
# Scale integral, we expect 24*3600=86400 evts/day * (period_days days)
print("Expect   : ", secinday * t1)
print("Integral : ", intgrl[-1])
_ = plt.plot(t, intgrl / 1e7, lw=2, label="integral/1e7")

# Sample from whole range and scale normed hist with time scale to match rate
nsam = int(1e4)
sam = sinfun.sample(t=t0, trange=[t0, t1*secinday], pars=sinpars,
                      n_samples=nsam)
h, b = np.histogram(sam, range=[t0, t1], bins=50, density=True)
m = get_binmids([b])[0]
_ = plt.hist(m, bins=b, weights=h * (t1 - t0), color="C0",
             alpha=0.5, label="sampled")

# Finally fit the sampled points again
runtime = (t1 - t0)
p0 = None  # Test default args
bf_pars = constfun.fit(t=m, rate=h * (t1 - t0), rate_std=None, p0=p0)
yfit = constfun.fun(t, bf_pars)
_ = plt.plot(t, yfit, lw=2, color="C3", ls="--", label="fitted")
p0 = constfun._get_default_seed(t=m, rate=h * (t1 - t0),
                                rate_std=np.ones_like(m))
yseed = constfun.fun(t, p0)
_ = plt.plot(t, yseed, lw=2, color="C3", ls="-", alpha=0.3,
             label="default seed")

plt.xlabel("time in MJD")
plt.ylabel("rate in Hz")
_ = plt.ylim(0, None)
plt.legend()
plt.tight_layout()

# plt.savefig("data/figs/rate_function_test.png", dpi=200)

plt.show()

### NonRateFunction

In [ ]:
# Just check that no errors are thrown and how fast it is
nonfun = RateFunc.NonRateFunction()
bf_pars = nonfun.fit(t=m, rate=h * (t1 - t0))
%timeit nonfun.fun(t, bf_pars)

## LLH

Test the LLH module.

It contains all functions for a specific LLH we want to use in our analysis.
Currently GRBLLH is implemented.

In [ ]:
sin_dec_bins = np.linspace(-1, 1, 50)

min_logE = 1  #  min(np.amin(_exp["logE"]), np.amin(mc["logE"]))
max_logE = 10 #  max(np.amax(_exp["logE"]), np.amax(mc["logE"]))
logE_bins = np.linspace(min_logE, max_logE, 40)

spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}

energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}

time_pdf_args = {"nsig": 4., "sigma_t_min": 2., "sigma_t_max": 30.}

grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

### Time PDF Ratio

Reproduce the paper plot.

Note that we get the PDFs for all srcs at once.
Their times are just all the same here.

In [ ]:
# Make a plot with ratios for different time windows as in the paper
# dt from t0 in seconds, clip at 4 sigma
dts = [[-1, 5], [-5, 50], [-20, 200]]
nsrcs = len(dts)
nsig = 4

# Arbitrary start date from data
t0 = np.repeat(np.random.choice(exp["timeMJD"], size=1),
               repeats=nsrcs).reshape(nsrcs, 1)
t0_sec = t0[0] * secinday  # Only single number needed, t0s are all equal

# Make t values for plotting in MJD around t0, to fit all in one plot
max_dt, min_dt = np.amax(dts), np.amin(dts)
dt_tot = max_dt - min_dt
clip = np.clip(dt_tot, 2, 30) * nsig
plt_range = np.array([min_dt - clip, max_dt + clip])
t = np.linspace(t0_sec + 1.2 * plt_range[0],
                t0_sec + 1.2 * plt_range[1], 1000) / secinday
_t = t * secinday - t0 * secinday

# Mark event time
plt.axvline(0, 0, 1, c="k", ls="--", lw=2, alpha=0.8)

# Get all at once
SoB = grbllh._soverb_time(t=t, src_t=t0, dt=dts)

colors = ["C0", "C3", "C2"]
for i in range(len(SoB)):
    # Plot seperately to give colors and labels
    plt.plot(_t[i], SoB[i], lw=2, c=colors[i],
             label=r"$T_\mathrm{{uni}}$: {:>3d}s, {:>3d}s".format(*dts[i]))
    # Fill uniform part, might look nicely
    # fbtw = (_t > 0) & (_t < dt)
    # plt.fill_between(_t[fbtw], 0, SoB[fbtw], color="C7", alpha=0.1)

# Plot stacked
weights = np.ones(nsrcs).reshape(nsrcs, 1) / nsrcs
stacked = np.sum(SoB * weights, axis=0) 
plt.plot(_t[0], stacked, ls="--", c="k", lw=2, label="stacked", alpha=0.8)
    
# Make it look like the paper plot, but with slightly extended borders
plt.xlim(1.2 * plt_range)
plt.ylim(0, np.amax(SoB) * 1.05)
plt.xlabel("t - t0 in sec")
plt.ylabel("S / B")
plt.legend(loc="upper right")
plt.grid(ls="--", lw=1)

# plt.savefig("./data/figs/time_pdf_ratio.png", dpi=200)

plt.show()

Get the injection time window.
This is needed for the injector, so only events in regions with non-zero PDF are injected.

In [ ]:
grbllh.get_injection_trange(t0, dts)

In [ ]:
# Compare manually
dts = np.array([[-1, 5], [-5, 50], [-20, 200]], dtype=np.float)
nsig, sig_min, sig_max = time_pdf_args.values()  # Beware if order is wrong :P
clip = np.clip(np.diff(dts, axis=1), sig_min, sig_max) * nsig
dts[:, 0] -= clip.reshape(len(dts))  # Same as flatten()
dts[:, 1] += clip.flatten()

dts

### Spatial background spline

This is the same technique as used in skylab, but with an extra step of adding the outermost bin edges to the spline gridpoints.
This way, the spline behaves reasonable at the edges and doesn't overshoot.

We could extend this by using the KDE integrated over every variable and then fitting a spline to that.
Or we could sample from the KDE and bin finely and fit a splien again.

For now we leave only the option to use data directly.
The spline fit is depending on the binning anyway.
Only the finely binned KDE version could resolve that issue.

In [ ]:
sin_dec = np.linspace(-1.05, 1.05, 200)
y = np.exp(grbllh._spatial_bg_spl(sin_dec))
_ = plt.hist(np.sin(_exp["dec"]), bins=50, normed=True)
plt.plot(sin_dec, y, lw=2)

### Spatial background pdf

Should be identical to calling the spline directly, except that the BG PDF is normalized to the whole sphere.
So we multiply the values by 2pi to account for that.

Here we see the difference to just calling the spline directly: The PDF is zero outside the definition range, the spline extrapolated.

In [ ]:
_ = plt.hist(np.sin(_exp["dec"]), bins=grbllh.spatial_pdf_args["bins"],
             normed=True)
sin_dec = np.linspace(-1.05, 1.05, 200)
y = 2 * np.pi * grbllh._pdf_spatial_background(ev_sin_dec=sin_dec)
plt.plot(sin_dec, y, lw=2)

### Spatial signal PDF

Compare signal and BG pdf.

First we create multiple sources and a single event and scan the event PDF by moving the event along the declination axis.
All PDFs have the height, because the same sigma is used.

Note that BG is here usually very small compared to the signal, because we sample the ev positions within 1 sigma around the source.

In [ ]:
LOG = False

nsrcs = 4
# Choose the event sigma from data
ev_sigma = np.random.choice(_exp["sigma"], size=1)

# Make nsrcs, same ra, but different dec. decs are distributed uniformly in
# the range of the largest sigma from the events (for illustration only)
src_dec = np.random.uniform(-ev_sigma, ev_sigma, size=nsrcs)
src_ra = np.ones_like(src_dec) * np.pi
plt_rnge = [np.amin(src_dec) - ev_sigma, np.amax(src_dec) + ev_sigma]

# Scan signal PDF for event declination
ev_dec = np.sin(np.linspace(plt_rnge[0], plt_rnge[1], 200))
ev_sin_dec = np.sin(ev_dec)
ev_ra = src_ra[0] * np.ones_like(ev_sin_dec)
ev_sig = np.ones_like(ev_sin_dec) * ev_sigma

# y has shape (nsrcs, nevts), where nevts are the ev_sin_dec values here (scan)
y = grbllh._pdf_spatial_signal(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sig)

if LOG:
    y = np.log10(y)

plt.plot(ev_dec, y.T, lw=2)
plt.vlines(src_dec, 0, np.amax(y), color="C7", linestyles="--", lw=2,
           label="srcs pos")

# Plot BG PDF to compare
bg = grbllh._pdf_spatial_background(ev_sin_dec=ev_sin_dec)
plt.plot(ev_dec, bg, lw=2, label="BG")

plt.xlim(*plt_rnge)
if LOG:
    plt.ylim(1e-5, 1.1 * np.amax(y))
else:
    plt.ylim(0, 1.1 * np.amax(y))
    
    
plt.xlabel("dec")
plt.ylabel("PDF per src")
plt.legend()
    
plt.tight_layout()

Here we use multiple events with different sigmas and scan again in declination by moving a single possible src position.
We get different heights, because of the different sigmas.

The PDFs each peak where the event position is.
If we had a single source, we would just read off the values at that position.

In [ ]:
LOG = True
# Make nevts, same ra, but different dec. sigmas chosen from data
nevts = 4
ev_sigma = np.random.choice(_exp["sigma"], size=nevts)
# Sample some evt decs uniformly around the horizon with spread of the largest 
# sigma to get some variation
max_sig = np.amax(ev_sigma)
ev_dec = np.random.uniform(-max_sig, max_sig, size=nevts)
ev_sin_dec = np.sin(ev_dec)
ev_ra = np.ones_like(ev_dec) * np.pi

# Plot margin PDF scanned for each src position for each event position
src_dec = np.linspace(-2. * max_sig, 2 * max_sig, 200)
src_ra = ev_ra[0] * np.ones_like(src_dec)

# This has shape (nsrcs, nevts)
y = grbllh._pdf_spatial_signal(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sigma)

if LOG:
    y = np.log10(y)

plt.plot(src_dec, y, lw=2)

plt.vlines(ev_dec, 0, np.amax(y) * 1.1, color="C7",
           linestyles="--", label="evts pos")

# Plot BG PDF to compare
bg = grbllh._pdf_spatial_background(ev_sin_dec=np.sin(src_dec))
plt.plot(src_dec, bg, lw=2, label="BG")

plt.xlim(src_dec[[0, -1]])
if LOG:
    plt.ylim(1e-5, 1.1 * np.amax(y))
else:
    plt.ylim(0, 1.1 * np.amax(y))
    
plt.legend()
plt.tight_layout()

### Spatial PDF ratio

In [ ]:
def plot_dec_vs_signal(S, ev_dec, src_ra, src_dec, weights, ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    # Plot signal per source for each event
    for i, (sra, sdec) in enumerate(zip(src_ra, src_dec)):
        ax.plot(np.rad2deg(ev_dec), S[i], ls="-")
        ax.plot(np.rad2deg(sdec), -10, "k|")

    # Simulate a simple stacking, one weight per source
    ax.plot(np.rad2deg(ev_dec), np.sum(weights * S, axis=0) / np.sum(weights),
             ls="--", c=dg, label="stacked")

    ax.set_xlim([-1 + smin, smax + 1])
    ax.set_xlabel("DEC in °")
    ax.set_ylabel("Signal pdf")
    ax.legend(loc="upper right")
    return ax

We make 4 plots to test everything:

1. [Top left] We place densely packed srcs at the declination range and scan the PDFs by varying the event declinations.
   Sigma is fixed to 1 for illustration.
   We expect just a row of gaussians along the dec range.
   The stacked signal is the weighted sum of all signal contributions at a single event dec position.
   
2. [Bottom left] We plot just the background PDF and its inverse for the dec range.
   The inverse PDF is what modulates the signal PDF.
   
3. [Top right] This modulation can be seen in this plot.
   It is basically the same as the first one, but now it's signal over background.
   So the signal peaks are modulated with the inverse BG PDF.
   
4. [Bottom right] This is the same plot as the third one, but this time we use the real data declination values instead of nicely spaced ones.
   The effect is the same but not reall visible, because each event has a different sigma, so the PDFs all have different heights and widths.
   It becomes more similar when using an 1° sigma for all events (just comment that line in).

In [ ]:
# Make srcs across the dec range. The hull of SoB should be shaped like the
# 1/(sinDec BG distribtuion). With a single source we couldn't see that,
# because it drops to zero far from the src position
smin, smax, step = -90, +90, 10
src_ra = np.deg2rad(np.arange(smin, smax + step, step))
src_dec = np.deg2rad(np.arange(smin, smax + step, step))

# Scan in dec by varying the evts dec
ev_ra = np.deg2rad(np.linspace(smin, smax, 1000))
ev_dec = np.deg2rad(np.linspace(smin, smax, 1000))
ev_sin_dec = np.sin(ev_dec)
ev_sig = np.deg2rad(np.ones_like(ev_ra))

# Some pseudo weights to simulate stacking
weights = np.arange(1, len(src_dec) + 1)[:, np.newaxis]

fig, ((axtl, axtr), (axbl, axbr)) = plt.subplots(2, 2, figsize=(12, 10))

# Signal only
S = grbllh._pdf_spatial_signal(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sig)
_ = plot_dec_vs_signal(S, ev_dec, src_ra, src_dec, weights, ax=axtl)
axtl.set_xlim(-90, 90)

# Background only
bins = grbllh.spatial_pdf_args["bins"]
h, b = np.histogram(np.sin(_exp["dec"]), bins=bins, density=True)
m = 0.5 * (b[:-1] + b[1:])
_ = axbl.hist(m, bins=bins, weights=h / 2 / np.pi, alpha=0.5)
_sin_dec = np.linspace(-1, 1, 1000)
bg_pdf = grbllh._pdf_spatial_background(_sin_dec)
axbl.plot(_sin_dec, bg_pdf, lw=2, label="pdf")
axbl.set_ylim(0, 0.2)
# 1 / BG PDF on second axis
axbl2 = axbl.twinx()
axbl2.plot(_sin_dec, 1. / bg_pdf, c="C2", lw=2, ls="--", label="1/pdf")
axbl2.set_ylim(0, (1 / bg_pdf).max())
axbl.set_xlabel("sinus DEC")
axbl.set_xlim(-1, 1)
axbl.legend(loc="upper left")
axbl2.legend(loc="upper center")

# SoB on example + BG PDF
SoB = grbllh._soverb_spatial(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sig)
weights = np.arange(1, len(src_dec) + 1)[:, np.newaxis]
_ = plot_dec_vs_signal(SoB, ev_dec, src_ra, src_dec, weights, ax=axtr)
axtr.plot(np.rad2deg(np.arcsin(_sin_dec)), bg_pdf, lw=3, label="BG pdf", c=dg)
axtr.set_xlim(-90, 90)
axtr.set_yscale("log")
axtr.set_ylim(np.amin(bg_pdf), 1e5)
axtr.legend(loc="upper left")

# Now with the real data. Sort first in dec to show with nice lines + BG PDF
idx = np.argsort(exp["dec"])
ev_ra = exp["ra"][idx]
ev_dec = exp["dec"][idx]
ev_sin_dec = np.sin(ev_dec)
ev_sig = exp["sigma"][idx]
# Comment in to match the simple example (all events have sigma 1°)
# ev_sig = np.deg2rad(np.ones_like(ev_ra))
SoB = grbllh._soverb_spatial(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sig)

_ = plot_dec_vs_signal(SoB, ev_dec, src_ra, src_dec, weights, ax=axbr)
axbr.plot(np.rad2deg(np.arcsin(_sin_dec)), bg_pdf,
          lw=3, label="BG pdf", c="C0")
axbr.set_yscale("log")
axbr.set_ylim(np.amin(bg_pdf), 1e5)
axbr.legend(loc="upper left")

plt.show()

### Energy ratio spline

This is the creation of the signal over background ratio for the energy PDF.
It is resolved in sinDec and logE to account for different positions on the sky and energies.

Missing values, where no data or MC is present is filled with interpolation values, conttrolled by the "fillval" option.

In [ ]:
fig, (al, ar) = plt.subplots(1, 2, figsize=(14,5))

energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}
grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

# Ratio spline with 'col' filling
x = np.linspace(-1.1, 1.1, num=1000 + 1)
y = np.linspace(0.5, 10.5, num=1000 + 1)
XX, YY = np.meshgrid(x, y)
xx, yy = map(np.ravel, [XX, YY])
gpts = np.vstack((xx, yy)).T
zz = np.exp(grbllh._energy_spl(gpts))
ZZ = zz.reshape(XX.shape)
# Plotting with hist creates strange effects... Use pcolormesh instead
img = al.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1e-3, vmax=1e3)
al.set_title("Spline interpolation: 'col'")
plt.colorbar(ax=al, mappable=img)

# With 'minmax' filling. Note: The small values in the lower row are due to
# plotting in log. We interpolate in linear space, so in log, the jump is
# very steep for small values.
energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "minmax", "interpol_log": False}
grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

zz = np.exp(grbllh._energy_spl(gpts))
ZZ = zz.reshape(XX.shape)
img = ar.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1e-3, vmax=1e3)
ar.set_title("Spline interpolation: 'minmax'")
plt.colorbar(ax=ar, mappable=img)

fig.tight_layout()

### Energy PDF ratio

Here we see again the difference to the direct spline evaluation.
The ratio function set's values outside to zero probability.

In [ ]:
fig, (al, ar) = plt.subplots(1, 2, figsize=(14,5))

energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": True}
grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

# Ratio spline with 'col' filling
x = np.linspace(-1.1, 1.1, num=1000 + 1)
y = np.linspace(0.5, 10.5, num=1000 + 1)
XX, YY = np.meshgrid(x, y)
xx, yy = map(np.ravel, [XX, YY])
gpts = np.vstack((xx, yy)).T
zz = grbllh._soverb_energy(xx, yy)
ZZ = zz.reshape(XX.shape)
# Plotting with hist creates strange effects... Use pcolormesh instead
img = al.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1e-3, vmax=1e3)
al.set_title("Spline interpolation: 'col'")
plt.colorbar(ax=al, mappable=img)

# With 'minmax' filling. Note: The small values in the lower row are due to
# plotting in log. We interpolate in linear space, so in log, the jump is
# very steep for small values.
energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "minmax", "interpol_log": True}
grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

zz = grbllh._soverb_energy(xx, yy)
ZZ = zz.reshape(XX.shape)
img = ar.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1e-3, vmax=1e3)
ar.set_title("Spline interpolation: 'minmax'")
plt.colorbar(ax=ar, mappable=img)

fig.tight_layout()

### Detector source weights

We use the same spline method to create a spline describing the sinDec dependence of a signal MC weighted to a specific astrophysical flux modell (usually unbroken power law).

Depending on the src position, we expect more or less signal from that src.
This is equivalent to folding with the detector exposure function.

Our stacking form is described by a multi position search where the signal term gets modified to:

$$
    S^\text{tot} = \sum_{j=1}^{N_\text{srcs}} w_j S_{ij} \quad\text{with}\quad
    \sum_j w_j = 1 \quad\text{with}\quad w_j = w_j^\text{theo}\cdot w_j^\text{det}
$$

The weights are a combination of the exposure weights and a-priori fixed intrinsic source weights, eg. from a known gamma flux.

In [ ]:
# Small hack to change the gamma without recreating the grbllh object
gamma_override = 2.13

grbllh.energy_pdf_args["gamma"] = gamma_override
mc_sin_dec = np.sin(mc["dec"])
mc_bins = grbllh.energy_pdf_args["bins"][0]
mc_dict = {"trueE": mc["trueE"], "ow": mc["ow"]}

grbllh._spatial_signal_spl = grbllh._create_sin_dec_spline(
    sin_dec=mc_sin_dec, bins=mc_bins, mc=mc_dict)

sin_dec = np.linspace(-1.05, 1.05, 200)
y = np.exp(grbllh._spatial_signal_spl(sin_dec))

# MC needs proper weighting
gamma = grbllh.energy_pdf_args["gamma"]
mc_w = mc["ow"] * mc["trueE"]**(-gamma)
mc_bins = energy_pdf_args["bins"][0]
h, b = np.histogram(np.sin(mc["dec"]), bins=mc_bins, weights=mc_w, normed=True)

# Smooth it, charge it, odd it, quick truncate it
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html
m = get_binmids([b])[0]
redux = 10  # Window len is nearest odd number to (number of bins / redux)
window_len = int(2 * np.floor((len(b) / redux) / 2) + 1)  # Must be odd
_h = scsignal.savgol_filter(h, window_len, 3, mode="mirror")
plt.hist(m, bins=b, weights=_h, histtype="step", lw=2, color="C1", ls="--")
plt.hist(m, bins=b, weights=h, histtype="step", lw=2, color="C3")

# Plot spline (fitted to unsmoothed)
plt.plot(sin_dec, y, lw=2, color="C2")

# Get weights for some srcs
src_sin_dec = np.linspace(-1, 1, 11)
src_dec = np.arcsin(src_sin_dec)
src_w_theo = np.ones_like(src_dec)
w = grbllh.get_src_weights(src_dec=src_dec, src_w_theo=src_w_theo)

# Revoke norm for plotting to see if weights are on the curve
src_dec_w = np.exp(grbllh._spatial_signal_spl(src_sin_dec))
_w = w * np.sum(src_dec_w * src_w_theo)

plt.plot(src_sin_dec, _w, "wo", ms=7, mew=1.5, mec="k")
plt.vlines(np.sin(src_dec), 0, 1.05 * np.amax(y), colors="C7",
           lw=1, linestyles="--")

plt.xlabel("sin(dec)")
plt.ylabel("PDF")
plt.title("$\gamma = {:.1f}$".format(gamma))

plt.xlim(-1, 1)
plt.ylim(0, 1.05 * np.amax(y))
plt.tight_layout()

print(w)
print(w.sum())

Here we just use some ascending theoretical weights in both directions.

- w1 should resemble the sindec curve from above
- w2 should rise overall to the right (less steep or reversed from w1)
- w3 should fall overall to the right (steeper than w1)

In [ ]:
_src_dec = np.arcsin(np.linspace(-1, 1, 21))

# Compare for different theoretical weights
src_w_theo = np.ones_like(_src_dec)
w1 = grbllh.get_src_weights(src_dec=_src_dec, src_w_theo=src_w_theo)

src_w_theo = np.arange(len(_src_dec)) + 1
w2 = grbllh.get_src_weights(src_dec=_src_dec, src_w_theo=src_w_theo)

src_w_theo = (np.arange(len(_src_dec)) + 1)[::-1]
w3 = grbllh.get_src_weights(src_dec=_src_dec, src_w_theo=src_w_theo)

plt.plot(np.sin(_src_dec), w1, "o", label="theo = 1 (orig)")
plt.plot(np.sin(_src_dec), w2, "o", label="theo = arange")
plt.plot(np.sin(_src_dec), w3, "o", label="theo = arange[::-1]")
plt.legend(loc="upper left")

Check if weights are the same when using event density (skylab style) instead.
It should make no difference because the weights are normalized anyway.

Note: The normalization and the pivot will not be included in the end, because they are constant for every source and for every dataset/year, so they get normalized out anyway.

In [ ]:
# Weight with numu diffuse 6yr flux norm, but same index as above
index = gamma_override
norm = 0.9 * 1e-18  # (GeV s sr cm^2)^-1, valid at 100 TeV = 1e5 GeV
pivot = 1e5
flux = norm * (mc["trueE"] / pivot)**(-index)
mc_w = mc["ow"] * flux * livetime * secinday
mc_bins = energy_pdf_args["bins"][0]

density = True
h, b = np.histogram(np.sin(mc["dec"]), bins=mc_bins, weights=mc_w,
                    density=density)

# Normalize (same as density=True)
if not density:
    h /= np.diff(b) * np.sum(h)

# PDF * Number of total events = Event densitiy
_h = h * mc_w.sum()  

mids = get_binmids([mc_bins])[0]
_ = plt.hist(mids, bins=mc_bins, weights=_h)

plt.xlabel("sindec")
plt.ylabel("Event density Nevts / sindec")

# Total events by integrating _h: Ntot = sum_i (_h_i * diff(bins)_i)
plt.title("Gamma = {:.2f}: Ntot = {:.2f}".format(
        index, np.sum(_h * np.diff(mc_bins))))

# Quick check the weight, no spline involved, directly use hist vals, so the
# Weights are not 100% percent equal. But equal enough to confirm
# Hack to include the first data point, as digitize is exclusive...
src_sin_dec[0] = -0.99
idx = np.digitize(src_sin_dec, bins=mc_bins, right=True) - 1
src_w_dec = _h[idx]
src_w_theo = np.ones_like(src_dec)
src_w = src_w_dec * src_w_theo / np.sum(src_w_dec * src_w_theo)

plt.plot(src_sin_dec, src_w_dec, "wo", ms=7, mew=1.5, mec="k")
plt.tight_layout()

print("Spline from PDF only")
print(w)

print("\nSpline from event density with livetime")
print(src_w.reshape(len(src_w), 1))

print("\nRatio")
print(w / src_w.reshape(len(src_w), 1))

### ln-LLH ratio

Plot llh and gradient.
The gradient is calculated analytically.
With this test, we simply want to check, if the gradient is OK and the likelihood behaves correctly.

First with only one source.

Note: We test here "super-signal-like" events. Every event is exactly at the src position and every time is exactly in the rime window, where the ration is max- Only the energy is distributed as background.
So only for really large time windows (really large) which have insanely high background rates we drop lower than the injected ns in our prediction.
This is because the background term can only counter background-like events, which have a low signal over background ratio.
For the events injected here, this rate is super high, so we always "fit" the exact amount of injected events.

For such a setup for each event SoB is equal.
So the gradient is zero at:

\begin{align}
    0 &= -1 + \sum_i \frac{S}{n_b B}\cdot \frac{1}{n_s \frac{S}{n_b B} + 1}
       = -1 + N \frac{S}{n_b B}\cdot \frac{1}{n_s \frac{S}{n_b B} + 1} \\
    \Leftrightarrow \frac{1}{N} &= \frac{1}{n_s + \frac{n_b B}{S}} \\
    \Leftrightarrow N &= n_s + \frac{n_b B}{S}
\end{align}

So now if the signal is super large (and that's what we ensured by using our super-signal-like events) the term $\frac{n_b B}{S} \rightarrow 0$ and we get $\hat{n}_S = N$ which is exactly what we observe.

Only if we set super high $n_B$, our $\hat{n}_S$ shrinks as $\frac{n_b B}{S}$ gets larger and larger and in the end $\hat{n}_S$ even turns negative, when the 1 / SoB ratio is larger than N.

In [ ]:
# Make up some setup
src_t = np.random.choice(_exp["timeMJD"], size=1)
dt = np.array([-20, 200])
# Expected background with rate 5mHz, kind of realistic
nb = 0.005 * np.diff(dt) * 1e6  # Uncomment to see ns shrink
src_ra = np.deg2rad([180])  # Arbitrarily placed single source
src_dec = np.deg2rad([10])
args = {"nb": nb, "src_t": src_t, "dt": dt,
        "src_ra": src_ra, "src_dec": src_dec,
        "src_w_theo": np.ones_like(src_dec)}

# Set the events artificially where the srcs are in space and nicely spaced
# times inside the search window, where time sob is large. Otherwise the llh
# is almost always peaked at 0
N = 10
mint, maxt = src_t + dt / secinday  # In MJD
timeMJD = np.linspace(mint, maxt, N)
X = np.random.choice(_exp, size=N)  # Only to copy the recarray structure
X["timeMJD"] = timeMJD
X["ra"] = np.ones_like(timeMJD) * src_ra
X["sinDec"] = np.ones_like(timeMJD) * np.sin(src_dec)
X["sigma"] = np.deg2rad(np.ones_like(timeMJD))

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
lnllh = np.empty(n_ns)
lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    lnllh[i], lnllh_grad[i] = grbllh.lnllh_ratio(X, theta, args)

# Manual "fit" by scanning the maximum
ns_max = ns[np.argmax(lnllh)]

plt.plot(ns, lnllh)
plt.xlim(xmin, xmax)
plt.ylim(0, 1.05 * np.amax(lnllh))
plt.axvline(ns_max, 0, 1, ls="--", lw=2, color="C7")
plt.title("nb = {:.2f}. ns max = {:.2f}".format(*nb, ns_max))
plt.show()

plt.plot(ns, lnllh_grad)
plt.axhline(0, 0, 1, ls="--", lw=2, color="C7")
plt.axvline(ns_max, 0, 1, ls="--", lw=2, color="C7")
plt.xlim(xmin, xmax)
plt.ylim(-5, 5)
plt.show()

This time we use multiple sources, but all at the exact same location and with the exact same properties.
We expect the very same result as in the single source case above, because the weighted sum of the signal terms reduces to

\begin{align}
    S^\text{tot} &= \sum_{j=1}^{N_\text{srcs}} w_j S_{ij}
                 = S_{i} \sum_{j=1}^{N_\text{srcs}} \frac{1}{N}
                 = S_i \\
    \Lambda &= -2\ln\left(\frac{\mathcal{L}_0}{\mathcal{L}_1}\right)
             = -n_S + \sum_{i=1}^N\ln\left(\frac{n_S S^\text{tot}}{\langle n_B\rangle B_i} + 1\right)
             = -n_S + \sum_{i=1}^N\ln\left(\frac{n_S S_i}{\langle n_B\rangle B_i} + 1\right)
\end{align}

as all signal terms are exaxtly the same and no further background locations are introduced.

In [ ]:
# Repeat sources exactly as the single one from above
nsrcs = 5
_src_t = np.repeat([src_t,], repeats=nsrcs, axis=0)
_dt = np.repeat(dt.reshape(1, 2), axis=0, repeats=nsrcs)
# Attention here: 100% overlapping windows so total BG is unchanged. To work
# in the stacking framework, we just split the expectation equally
_nb = 0.005 * np.diff(_dt) / nsrcs * 1e6  # Uncomment to see ns shrink

_src_ra = np.repeat([src_ra,], repeats=nsrcs, axis=0)
_src_dec = np.repeat([src_dec,], repeats=nsrcs, axis=0)

_args = {"nb": _nb, "src_t": _src_t, "dt": _dt,
         "src_ra": _src_ra, "src_dec": _src_dec,
         "src_w_theo": np.ones_like(_src_dec)}

# Also use the very same events for all sources here
_X = np.copy(X)

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
_lnllh = np.empty(n_ns)
_lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    _lnllh[i], _lnllh_grad[i] = grbllh.lnllh_ratio(_X, theta, _args)

# Manual "fit" by scanning the maximum
_ns_max = ns[np.argmax(_lnllh)]

plt.plot(ns, _lnllh)
plt.xlim(xmin, xmax)
plt.ylim(0, 1.05 * np.amax(_lnllh))
plt.axvline(_ns_max, 0, 1, ls="--", lw=2, color="C7")
plt.title("nb = {:.2f} per source. ns max = {:.2f}".format(*_nb[0], _ns_max))
plt.show()

plt.plot(ns, _lnllh_grad)
plt.axhline(0, 0, 1, ls="--", lw=2, color="C7")
plt.axvline(_ns_max, 0, 1, ls="--", lw=2, color="C7")
plt.xlim(xmin, xmax)
plt.ylim(-5, 5)
plt.show()

Now the almost same thing, but with changed right ascensions only.
We distribute them equally around a fixed declination.
Also everything else is left as before.

This case is a bit more tricky, but a combination of the two cases above makes sure we still fit N events.

We inject the same number of events (N) but we get nsrcs times the BG (because the windows don't overlap anymore).
Each event only contributes to the window where it spatially is placed, so per source only N / nsrcs events (we choosed them so the number distribute nicely) have a SoB > 0.

This means, that the total signal term is reduced by a factor of nsrcs, as the zero signal terms can't compensate the unaffected backgound which is still the same for all events.

So even though our stacked signal term is reduced by a factor of nsrcs  we still get the same fit result, because the signal term is still huge and we still satisfy the condition $\frac{n_b B}{S}\rightarrow 0$.

But we need slightly less cranked up background rate to let the best fit ns shrink as in the previous cases.

In [ ]:
# Repeat sources exactly as the single one from above
nsrcs = 5
_src_t = np.repeat([src_t,], repeats=nsrcs, axis=0)
_dt = np.repeat(dt.reshape(1, 2), axis=0, repeats=nsrcs)
# Windows don't overlap anymore, so use full BG for each window
_nb = 0.005 * np.diff(_dt) * 1e6  # Test this to see ns shrink

# Handpick to let windows not overlap
_src_ra = np.deg2rad([0, 30, 60, 90, 120]).reshape(nsrcs, 1)
_src_dec = np.repeat([src_dec,], repeats=nsrcs, axis=0)

_args = {"nb": _nb, "src_t": _src_t, "dt": _dt,
         "src_ra": _src_ra, "src_dec": _src_dec,
         "src_w_theo": np.ones_like(_src_dec)}

# We used 5 srcs and 10 events, so we just repeat the ras once
# This is not very obvious on how to scale to arbirary Ns and nsrcs
# I'm not very sure here, how many events to inject to exactly match the cases
# above.
# Here we just have 2 evts per window and still have ns of 10, even though
# signal should get donwweighted to 1/5 of the two cases above per source.
_X = np.copy(X)
_X["ra"] = np.repeat(_src_ra, repeats=2)

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
_lnllh = np.empty(n_ns)
_lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    _lnllh[i], _lnllh_grad[i] = grbllh.lnllh_ratio(_X, theta, _args)

# Manual "fit" by scanning the maximum
_ns_max = ns[np.argmax(_lnllh)]

plt.plot(ns, _lnllh)
plt.xlim(xmin, xmax)
plt.ylim(0, 1.05 * np.amax(_lnllh))
plt.axvline(_ns_max, 0, 1, ls="--", lw=2, color="C7")
plt.title("nb = {:.2f} per source. ns max = {:.2f}".format(*_nb[0], _ns_max))
plt.show()

plt.plot(ns, _lnllh_grad)
plt.axhline(0, 0, 1, ls="--", lw=2, color="C7")
plt.axvline(_ns_max, 0, 1, ls="--", lw=2, color="C7")
plt.xlim(xmin, xmax)
plt.ylim(-5, 5)
plt.show()

## Analysis

In [ ]:
gamma = energy_pdf_args["gamma"]
weights = mc["trueE"]**(-gamma) * mc["ow"]
_ = plt.hist(mc["sinDec"], bins=energy_pdf_args["bins"][0], weights=weights)

plt.xlabel("southern sky <- horizon -> northern sky")

### Fit LLH paramters

In [ ]:
def plot_llh(ns, lnllh, lnllh_grad, ns_max, xmin, xmax):
    fig, (al, ar) = plt.subplots(1, 2, figsize=(10, 4))
    al.plot(ns, lnllh)
    if ns_max == 0:
        al.set_xlim(-1, 1)
    else:
        al.set_xlim(xmin, 2 * ns_max)
    al.set_ylim(0, 1.05 * np.amax(lnllh))
    al.axvline(ns_max, 0, 1, ls="--", lw=2, color="C7")
    al.set_title("LLH")

    ar.plot(ns, lnllh_grad)
    ar.axhline(0, 0, 1, ls="--", lw=2, color="C7")
    ar.axvline(ns_max, 0, 1, ls="--", lw=2, color="C7")
    if ns_max == 0:
        al.set_xlim(-1, 1)
    else:
        al.set_xlim(xmin, 2 * ns_max)
    ar.set_ylim(-5, 5)
    ar.set_title("LLH gradient in ns")
    fig.tight_layout()
    return fig, (al, ar)

#### Single source

First we test if the module simply wrapps the LLH module correctly.
This should reproduce same results (not regarding random fluctuations of course) as in the section ln-llh ratio, as we test the same setup as above here.

In [ ]:
# First make a likelihood we want to test with
sin_dec_bins = np.linspace(-1, 1, 50)

min_logE = 1  #  min(np.amin(_exp["logE"]), np.amin(mc["logE"]))
max_logE = 10 #  max(np.amax(_exp["logE"]), np.amax(mc["logE"]))
logE_bins = np.linspace(min_logE, max_logE, 40)

spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}
energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}
time_pdf_args = {"nsig": 4.}

grbllh = LLH.GRBLLH(X=_exp, MC=mc,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

# Now we make a single source record array
dt = np.atleast_2d([[-20, 200],])
n_srcs = len(dt)
nb = 0.005 * np.diff(dt)  # Rate 5mHz, kind of realistic

dtype = [("t", np.float), ("dt0", np.float), ("dt1", np.float),
         ("ra", np.float), ("dec", np.float)]
srcs = np.zeros((n_srcs, ), dtype=dtype)
srcs["t"] = np.random.choice(_exp["timeMJD"], size=n_srcs)
srcs["dt0"] = dt[:, 0]
srcs["dt1"] = dt[:, 1]
srcs["ra"] = np.random.uniform(0, 2 * np.pi, n_srcs)
srcs["dec"] = np.arcsin(np.random.uniform(-1, 1, n_srcs))

# And manually put it in an arg list of dicts
args = n_srcs * [{}]
for i in range(n_srcs):
    args[i]["nb"] = nb[i]
    args[i]["dt"] = [srcs[i]["dt0"], srcs[i]["dt1"]]
    args[i]["src_t"] = srcs[i]["t"]
    args[i]["src_ra"] = srcs[i]["ra"]
    args[i]["src_dec"] = srcs[i]["dec"]

# Build the analysis module
ana = Analysis.TransientsAnalysis(srcs=srcs, llh=grbllh)

In [ ]:
# Place events directly where the srcs are, to have a chance to see the LLh
# take actual maximum values different from 0
N = 100
mint, maxt = srcs["t"] + dt[0] / secinday
timeMJD = np.linspace(mint, maxt, N)
X = np.random.choice(_exp, size=N)a
X["timeMJD"] = timeMJD
X["ra"] = np.ones_like(timeMJD) * srcs["ra"]
X["dec"] = np.ones_like(timeMJD) * srcs["dec"]

# Manual "fit" by scanning the maximum
n_ns = 1000
xmin, xmax = 0, 20  # Scan range, chosen after trial and error
ns = np.linspace(xmin, xmax, n_ns)
lnllh = np.zeros(n_ns)
lnllh_grad = np.zeros(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    for j in range(n_srcs):
        f, g = ana.llh.lnllh_ratio(X, theta, args[j])
        lnllh[i] += f
        lnllh_grad[i] += g

ns_max = ns[np.argmax(lnllh)]
plot_llh(ns, lnllh, lnllh_grad, ns_max, xmin, xmax)
plt.show()

In [ ]:
# Also let's quickly see, how the events are distributed within the PDF
dt = args[0]["dt"]
x = np.linspace(X["timeMJD"][0] + 6 * dt[0] / secinday,
                X["timeMJD"][-1] + 1 * dt[1] / secinday, 10 * N)
y = ana.llh._soverb_time(t=x, src_t=srcs["t"], dt=dt)
plt.plot(x, y)
plt.vlines(X["timeMJD"], 0, np.amax(y), colors="C7", linestyles="-",
           alpha=0.2, lw=0.5)
plt.show()

Test if we can do the same as above, but usign a real fitter this time to get the maximum.
The LLH and the maximum should be identical to the one above (except for small errors in scanning vs fitting ns).

In [ ]:
# Seed for the fitter
theta0 = {"ns": 1}
# Args must only contain "nb", everything else is used from self.srcs
args = []
for i in range(n_srcs):
    args.append({"nb": nb[i]})

res = ana.fit_lnllh_ratio_params(X, theta0, args)

plot_llh(ns, lnllh, lnllh_grad, res.x, xmin, xmax)
plt.show()

#### Multiple sources

We only have to add more sources and the code from aboove should work the same, with some broadcasting effort to setup the events.

In [ ]:
# First make a likelihood we want to test with
sin_dec_bins = np.linspace(-1, 1, 50)

min_logE = 1  #  min(np.amin(_exp["logE"]), np.amin(mc["logE"]))
max_logE = 10 #  max(np.amax(_exp["logE"]), np.amax(mc["logE"]))
logE_bins = np.linspace(min_logE, max_logE, 40)

spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}
energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}
time_pdf_args = {"nsig": 4.}

grbllh = LLH.GRBLLH(X=_exp, MC=mc,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

# Now we make a single source record array
dt = np.atleast_2d([[-20, 200], [-5, 20], [0, 300]])
n_srcs = len(dt)
nb = 0.005 * np.diff(dt)  # Rate 5mHz, kind of realistic

dtype = [("t", np.float), ("dt0", np.float), ("dt1", np.float),
         ("ra", np.float), ("dec", np.float)]
srcs = np.zeros((n_srcs, ), dtype=dtype)
srcs["t"] = np.random.choice(_exp["timeMJD"], size=n_srcs)
srcs["dt0"] = dt[:, 0]
srcs["dt1"] = dt[:, 1]
srcs["ra"] = np.random.uniform(0, 2 * np.pi, n_srcs)
srcs["dec"] = np.arcsin(np.random.uniform(-1, 1, n_srcs))

# And manually put it in an arg list of dicts
args = []
for i in range(n_srcs):
    args.append({"nb": nb[i]})
    args[i]["nb"] = nb[i]
    args[i]["dt"] = [srcs[i]["dt0"], srcs[i]["dt1"]]
    args[i]["src_t"] = srcs[i]["t"]
    args[i]["src_ra"] = srcs[i]["ra"]
    args[i]["src_dec"] = srcs[i]["dec"]

# Build the analysis module
ana = Analysis.TransientsAnalysis(srcs=srcs, llh=grbllh)

In [ ]:
# Place events directly where the srcs are, to have a chance to see the LLh
# take actual maximum values different from 0
N = 100
# Make time for each src window
mint, maxt = srcs["t"] + dt[:, 0] / secinday, srcs["t"] + dt[:, 1] / secinday
timeMJD = np.empty((n_srcs, N), dtype=np.float)
for j in range(n_srcs):
    timeMJD[j] = np.linspace(mint[j], maxt[j], N)
X = np.random.choice(_exp, size=(n_srcs, N))
X["timeMJD"] = timeMJD
X["ra"] = np.ones(N) * srcs["ra"].reshape(n_srcs, 1)
X["dec"] = np.ones(N) * srcs["dec"].reshape(n_srcs, 1)
X = X.flatten()

# Manual "fit" by scanning the maximum
n_ns = 1000
xmin, xmax = 0, 200  # Scan range, chosen after trial and error
ns = np.linspace(xmin, xmax, n_ns)
lnllh = np.zeros(n_ns)
lnllh_grad = np.zeros(n_ns)

# Weight ns with BG expectation to get proper ratios in the fit
weights = np.array([arg["nb"] for arg in args])
if np.sum(weights) > 0:
    weights /= np.sum(weights)
else:
    weights = np.ones_like(weights) / n_src
    
lnllh = np.zeros(n_ns, dtype=np.float)
lnllh_grad = np.zeros((n_ns, 1), dtype=np.float)
for i in range(n_ns):
    for j in range(n_srcs):
        theta = {"ns": ns[i] * weights[j]}
        f, g = ana.llh.lnllh_ratio(X, theta, args[j])
        lnllh[i] += f
        lnllh_grad[i] += weights[j] * g

ns_max = ns[np.argmax(lnllh)]
plot_llh(ns, lnllh, lnllh_grad, ns_max, xmin, xmax)
plt.show()

Again check using the class function

In [ ]:
# Seed for the fitter
theta0 = {"ns": 1}
# Args must only contain "nb", everything else is used from self.srcs
args = []
for i in range(n_srcs):
    args.append({"nb": nb[i]})
    
res = ana.fit_lnllh_ratio_params(X, theta0, args, bounds=None)
    
plot_llh(ns, lnllh, lnllh_grad, res.x, xmin, xmax)
plt.show()

### Trials

This pulls all from above together.

We need all injectors and the LLH.
Then we inject events from those injectors per trial and fit the LLH for each configuration.
We end up with a test statistic, modeling our null hypothesis.

#### BG only

In [ ]:
# Create a bg rat einjector model
def filter_runs(run):
    """
    Filter runs as stated in jfeintzig's doc.
    """
    exclude_runs = [120028, 120029, 120030, 120087, 120156, 120157]
    if ((run["good_i3"] == True) & (run["good_it"] == True) &
        (run["run"] not in exclude_runs)):
        return True
    else:
        return False

# Let's create an injector using a goodrun list.
runlist="data/runlists/ic86-i-goodrunlist.json"
rate_func = RateFunc.Sinus1yrRateFunction()
runlist_inj = BGRateInj.RunlistBGRateInjector(runlist, filter_runs, rate_func)

# `fit` the injector to make it usable
times = exp["timeMJD"]
rate_func = runlist_inj.fit(T=times, x0=None, remove_zero_runs=True)

In [ ]:
# Create our bg injector, here we use the data resampler
data_inj = BGInj.DataBGInjector()
data_inj.fit(sample)

In [ ]:
# Least we need a LLH, use the GRBLLH
sin_dec_bins = np.linspace(-1, 1, 50)

# Choose borders and bins by eye
min_logE = 1 
max_logE = 10
logE_bins = np.linspace(min_logE, max_logE, 40)

spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}
energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}
time_pdf_args = {"nsig": 4.}

grbllh = LLH.GRBLLH(X=_exp, MC=mc,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

In [ ]:
# Create some srcs to test for
dt = np.atleast_2d([[-20, 200], [-5, 20], [0, 300]])
n_srcs = len(dt)

dtype = [("t", np.float), ("dt0", np.float), ("dt1", np.float),
         ("ra", np.float), ("dec", np.float)]
srcs = np.zeros((n_srcs, ), dtype=dtype)
srcs["t"] = np.random.choice(_exp["timeMJD"], size=n_srcs)
srcs["dt0"] = dt[:, 0]
srcs["dt1"] = dt[:, 1]
srcs["ra"] = np.random.uniform(0, 2 * np.pi, n_srcs)
srcs["dec"] = np.arcsin(np.random.uniform(-1, 1, n_srcs))

In [ ]:
# And now the analysis object
ana = Analysis.TransientsAnalysis(srcs=srcs, llh=grbllh)

In [ ]:
self = ana

_X = []
times = []
rnd_ra = []
args = []

for src_idx in range(len(srcs)):
    # Samples times and thus number of bg expectated events
    t = self.srcs["t"][src_idx]
    dt = [self.srcs["dt0"][src_idx], self.srcs["dt1"][src_idx]]
    _times = runlist_inj.sample(t=t, trange=dt, ntrials=1)[0]
    nb = len(_times)
    times.append(_times)
    args.append({"nb": nb})
    # Sample rest of features
    if nb > 0:
        _X.append(data_inj.sample(n_samples=nb))
        rnd_ra.append(np.random.uniform(0, 2. * np.pi, size=nb))

names = ["ra", "sinDec", "timeMJD", "logE", "sigma"]
dtype = [(n, t) for (n, t) in zip(names, len(names) * [np.float])]
nb_tot = np.sum([d["nb"] for d in args])
X = np.empty((nb_tot, ), dtype=dtype)
# Make output array in compatible format
_X = flatten_list_of_1darrays(_X)
X["ra"] = flatten_list_of_1darrays(rnd_ra)
X["sinDec"] = np.sin(_X[:, 1])
X["logE"] = _X[:, 0]
X["sigma"] = _X[:, 2]
X["timeMJD"] = flatten_list_of_1darrays(times)

nb_tot

In [ ]:
ana.fit_lnllh_ratio_params(X, theta0={"ns": 1}, args=args)